In [1]:
import warnings
warnings.simplefilter('ignore')
import polars as pl
import os
import gc
import re
from collections import Counter

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', 300)
from tqdm.auto import tqdm
tqdm.pandas()
import polars as pl
import pandas as pd
import numpy as np
import re
from lightgbm import LGBMRegressor
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold, train_test_split,GridSearchCV
from scipy.stats import skew, kurtosis
import warnings

import copy
# from gensim.models import Word2Vec
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.metrics import mean_squared_error

import lightgbm as lgb
from scipy.stats import linregress
from scipy.stats import entropy

from scipy.stats import skew, kurtosis
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler, MinMaxScaler

import xgboost as xgb
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from sklearn.linear_model import Ridge, LinearRegression, Lasso, ElasticNet, LogisticRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.ensemble import VotingRegressor
from sklearn.datasets import make_regression

In [2]:
# def duplicated_feature(df):
#     print('delete dupilcate columns..')
#     df_temp = df.copy()
#     columns_dataFrame = df.columns
#     df = df.T.drop_duplicates().T
#     columns_df_temp = df.columns
#     final_columns_with_no_duplicate = [col for col in columns_dataFrame if col not in columns_df_temp]
#     df = df_temp.copy()
#     df = df.drop(final_columns_with_no_duplicate,axis=1)
#     gc.collect()
#     print()
#     return df

In [3]:
PATH = '/kaggle/input/linking-writing-processes-to-writing-quality/'

train_logs = pd.read_csv(f'{PATH}train_logs.csv')#[:200]
train_scores = pd.read_csv(f'{PATH}train_scores.csv')
test_logs = pd.read_csv(f'{PATH}test_logs.csv')

In [4]:
# TODO: reduce memory code

# Part 1 (kaggle comunity features)

In [5]:
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 
          'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
text_changes = ['q', ' ', 'NoChange', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']


def activity_counts(df):
    tmp_df = df.groupby('id').agg({'activity': list}).reset_index()
    ret = list()
    for li in tqdm(tmp_df['activity'].values):
        items = list(Counter(li).items())
        di = dict()
        for k in activities:
            di[k] = 0
        for item in items:
            k, v = item[0], item[1]
            if k in di:
                di[k] = v
        ret.append(di)
    ret = pd.DataFrame(ret)
    cols = [f'activity_{i}_count' for i in range(len(ret.columns))]
    ret.columns = cols
    return ret


def event_counts(df, colname):
    tmp_df = df.groupby('id').agg({colname: list}).reset_index()
    ret = list()
    for li in tqdm(tmp_df[colname].values):
        items = list(Counter(li).items())
        di = dict()
        for k in events:
            di[k] = 0
        for item in items:
            k, v = item[0], item[1]
            if k in di:
                di[k] = v
        ret.append(di)
    ret = pd.DataFrame(ret)
    cols = [f'{colname}_{i}_count' for i in range(len(ret.columns))]
    ret.columns = cols
    return ret


def text_change_counts(df):
    tmp_df = df.groupby('id').agg({'text_change': list}).reset_index()
    ret = list()
    for li in tqdm(tmp_df['text_change'].values):
        items = list(Counter(li).items())
        di = dict()
        for k in text_changes:
            di[k] = 0
        for item in items:
            k, v = item[0], item[1]
            if k in di:
                di[k] = v
        ret.append(di)
    ret = pd.DataFrame(ret)
    cols = [f'text_change_{i}_count' for i in range(len(ret.columns))]
    ret.columns = cols
    return ret


punctuations = ['"', '.', ',', "'", '-', ';', ':', '?', '!', '<', '>', '/',
                '@', '#', '$', '%', '^', '&', '*', '(', ')', '_', '+']
def match_punctuations(df):
    tmp_df = df.groupby('id').agg({'down_event': list}).reset_index()
    ret = list()
    for li in tqdm(tmp_df['down_event'].values):
        cnt = 0
        items = list(Counter(li).items())
        for item in items:
            k, v = item[0], item[1]
            if k in punctuations:
                cnt += v
        ret.append(cnt)
    ret = pd.DataFrame({'punct_cnt': ret})
    return ret


def get_input_words(df):
    tmp_df = df[(~df['text_change'].str.contains('=>'))&(df['text_change'] != 'NoChange')].reset_index(drop=True)
    tmp_df = tmp_df.groupby('id').agg({'text_change': list}).reset_index()
    tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: ''.join(x))
    tmp_df['text_change'] = tmp_df['text_change'].apply(lambda x: re.findall(r'q+', x))
    tmp_df['input_word_count'] = tmp_df['text_change'].apply(len)
    tmp_df['input_word_length_mean'] = tmp_df['text_change'].apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0))
    tmp_df['input_word_length_max'] = tmp_df['text_change'].apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0))
    tmp_df['input_word_length_std'] = tmp_df['text_change'].apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0))
    tmp_df.drop(['text_change'], axis=1, inplace=True)
    return tmp_df

In [6]:
def make_feats(df):
    
    # id
    feats = pd.DataFrame({'id': df['id'].unique().tolist()})
    
    for gap in [1, 2, 3, 5, 10, 20, 50, 100]:
    # time shift
        df['up_time_shift'] = df.groupby('id')['up_time'].shift(gap)
        df[f'action_time_gap_{gap}'] = df['down_time'] - df['up_time_shift']
        df.drop('up_time_shift', axis=1, inplace=True)

        # cursor position shift
        df['cursor_position_shift'] = df.groupby('id')['cursor_position'].shift(gap)
        df[f'cursor_position_change_{gap}'] = np.abs(df['cursor_position'] - df['cursor_position_shift'])
        df.drop('cursor_position_shift', axis=1, inplace=True)

        # word count shift
        df['word_count_shift'] = df.groupby('id')['word_count'].shift(gap)
        df[f'word_count_change_{gap}'] = np.abs(df['word_count'] - df['word_count_shift'])
        df.drop('word_count_shift', axis=1, inplace=True)
    
    # stats feats
    for item in tqdm([
        ('event_id', ['max']),
        ('up_time', ['max']),
        ('action_time', ['max', 'min', 'mean', 'std', 'quantile', 'sem', 'sum', 'skew', pd.DataFrame.kurt]),
        ('activity', ['nunique']),
        ('down_event', ['nunique']),
        ('up_event', ['nunique']),
        ('text_change', ['nunique']),
        ('cursor_position', ['nunique', 'max', 'quantile', 'sem', 'mean']),
        ('word_count', ['nunique', 'max', 'quantile', 'sem', 'mean']),
        ('action_time_gap_1', ['max', 'min', 'mean', 'std', 'sum']),
        ('action_time_gap_2', ['max', 'min', 'mean', 'std', 'sum']),
        ('action_time_gap_3', ['max', 'min', 'mean', 'std', 'sum']),
        ('action_time_gap_5', ['max', 'min', 'mean', 'std', 'sum']),
        ('action_time_gap_10', ['max', 'min', 'mean', 'std', 'sum']),
        ('action_time_gap_20', ['max', 'min', 'mean', 'std', 'sum']),
        ('action_time_gap_50', ['max', 'min', 'mean', 'std', 'sum']),
        ('action_time_gap_100', ['max', 'min', 'mean', 'std', 'sum']),
        ('cursor_position_change_1', ['max', 'mean', 'std', 'sum']),
        ('cursor_position_change_2', ['max', 'mean', 'std', 'sum']),
        ('cursor_position_change_3', ['max', 'mean', 'std', 'sum']),
        ('cursor_position_change_5', ['max', 'mean', 'std', 'sum']),
        ('cursor_position_change_10', ['max', 'mean', 'std', 'sum']),
        ('cursor_position_change_20', ['max', 'mean', 'std', 'sum']),
        ('cursor_position_change_50', ['max', 'mean', 'std', 'sum']),
        ('cursor_position_change_100', ['max', 'mean', 'std', 'sum']),
        ('word_count_change_1', ['max', 'mean', 'std', 'sum']),
        ('word_count_change_2', ['max', 'mean', 'std', 'sum']),
        ('word_count_change_3', ['max', 'mean', 'std', 'sum']),
        ('word_count_change_5', ['max', 'mean', 'std', 'sum']),
        ('word_count_change_10', ['max', 'mean', 'std', 'sum']),
        ('word_count_change_20', ['max', 'mean', 'std', 'sum']),
        ('word_count_change_50', ['max', 'mean', 'std', 'sum']),
        ('word_count_change_100', ['max', 'mean', 'std', 'sum']),
    ]):
        colname, methods = item[0], item[1]
        for method in methods:
            tmp_df = df.groupby(['id']).agg({colname: method}).reset_index().rename(columns={colname: f'{colname}_{method}'})
            feats = feats.merge(tmp_df, on='id', how='left')
    
    # counts
    tmp_df = activity_counts(df)
    feats = pd.concat([feats, tmp_df], axis=1)
    tmp_df = event_counts(df, 'down_event')
    feats = pd.concat([feats, tmp_df], axis=1)
    tmp_df = event_counts(df, 'up_event')
    feats = pd.concat([feats, tmp_df], axis=1)
    tmp_df = text_change_counts(df)
    feats = pd.concat([feats, tmp_df], axis=1)
    tmp_df = match_punctuations(df)
    feats = pd.concat([feats, tmp_df], axis=1)
    
    # input words
    tmp_df = get_input_words(df)
    feats = pd.merge(feats, tmp_df, on='id', how='left')
    
    # compare feats
    feats['word_time_ratio'] = feats['word_count_max'] / feats['up_time_max']
    feats['word_event_ratio'] = feats['word_count_max'] / feats['event_id_max']
    feats['event_time_ratio'] = feats['event_id_max']  / feats['up_time_max']
#     feats['idle_time_ratio'] = feats['action_time_gap_1'] / feats['up_time_max']
    
    return feats

In [7]:
train_feats = make_feats(train_logs)
test_feats = make_feats(test_logs)

  0%|          | 0/33 [00:00<?, ?it/s]

  0%|          | 0/2471 [00:00<?, ?it/s]

  0%|          | 0/2471 [00:00<?, ?it/s]

  0%|          | 0/2471 [00:00<?, ?it/s]

  0%|          | 0/2471 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:

# train_feats.to_csv('part_1_training.csv',index=False)
train_feats.shape

(2471, 190)

In [9]:
train_feats = train_feats.merge(train_scores, on='id', how='left')
train_feats.shape

(2471, 191)

In [10]:
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
up_events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', 'dot', 'comma', 
          'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "abostrove", 'Delete', 'Unidentified']

columns = [
    (
        (pl.col("down_time") - pl.col("up_time").shift(1))
        .over("id")
        .alias("time_diff")
    ),
    
    (
        (pl.col("cursor_position") - pl.col("cursor_position").shift(1))
        .over("id")
        .alias("cursor_position_diff")
    ),
    
    (
        (pl.col("word_count") - pl.col("word_count").shift(1))
        .over("id")
        .alias("word_count_diff")
    ),
]

def time_polar_process(x):
    aggs = [
        
        *[pl.col("time_diff").filter(pl.col("up_event") == c).std().alias(f"time_diff_{c}_std") for c in up_events],
        *[pl.col("time_diff").filter(pl.col("up_event") == c).mean().alias(f"time_diff_{c}_mean") for c in up_events],
        *[pl.col("time_diff").filter(pl.col("up_event") == c).sum().alias(f"time_diff_{c}_sum") for c in up_events],
        *[pl.col("time_diff").filter(pl.col("up_event") == c).median().alias(f"time_diff_{c}_median") for c in up_events],
        *[pl.col("time_diff").filter(pl.col("up_event") == c).max().alias(f"time_diff_{c}_max") for c in up_events],
        
        *[pl.col("cursor_position_diff").filter(pl.col("up_event") == c).std().alias(f"cursor_position_diff_{c}_std") for c in up_events],
        *[pl.col("cursor_position_diff").filter(pl.col("up_event") == c).mean().alias(f"cursor_position_diff_{c}_mean") for c in up_events],
        *[pl.col("cursor_position_diff").filter(pl.col("up_event") == c).sum().alias(f"cursor_position_diff_{c}_sum") for c in up_events],
        *[pl.col("cursor_position_diff").filter(pl.col("up_event") == c).median().alias(f"cursor_position_diff_{c}_median") for c in up_events],
        *[pl.col("cursor_position_diff").filter(pl.col("up_event") == c).max().alias(f"cursor_position_diff_{c}_max") for c in up_events],
        
        *[pl.col("word_count_diff").filter(pl.col("up_event") == c).std().alias(f"word_count_diff_{c}_std") for c in up_events],
        *[pl.col("word_count_diff").filter(pl.col("up_event") == c).mean().alias(f"word_count_diff_{c}_mean") for c in up_events],
        *[pl.col("word_count_diff").filter(pl.col("up_event") == c).sum().alias(f"word_count_diff_{c}_sum") for c in up_events],
        *[pl.col("word_count_diff").filter(pl.col("up_event") == c).median().alias(f"word_count_diff_{c}_median") for c in up_events],
        *[pl.col("word_count_diff").filter(pl.col("up_event") == c).max().alias(f"word_count_diff_{c}_max") for c in up_events],

        *[pl.col("time_diff").filter(pl.col("activity") == c).std().alias(f"time_diff_{c}_std") for c in activities],
        *[pl.col("time_diff").filter(pl.col("activity") == c).mean().alias(f"time_diff_{c}_mean") for c in activities],
        *[pl.col("time_diff").filter(pl.col("activity") == c).sum().alias(f"time_diff_{c}_sum") for c in activities],
        *[pl.col("time_diff").filter(pl.col("activity") == c).median().alias(f"time_diff_{c}_median") for c in activities],
        *[pl.col("time_diff").filter(pl.col("activity") == c).max().alias(f"time_diff_{c}_max") for c in activities],
        
        *[pl.col("cursor_position_diff").filter(pl.col("activity") == c).std().alias(f"cursor_position_diff_{c}_std") for c in activities],
        *[pl.col("cursor_position_diff").filter(pl.col("activity") == c).mean().alias(f"cursor_position_diff_{c}_mean") for c in activities],
        *[pl.col("cursor_position_diff").filter(pl.col("activity") == c).sum().alias(f"cursor_position_diff_{c}_sum") for c in activities],
        *[pl.col("cursor_position_diff").filter(pl.col("activity") == c).median().alias(f"cursor_position_diff_{c}_median") for c in activities],
        *[pl.col("cursor_position_diff").filter(pl.col("activity") == c).max().alias(f"cursor_position_diff_{c}_max") for c in activities],
        
        *[pl.col("word_count_diff").filter(pl.col("activity") == c).std().alias(f"word_count_diff_{c}_std") for c in activities],
        *[pl.col("word_count_diff").filter(pl.col("activity") == c).mean().alias(f"word_count_diff_{c}_mean") for c in activities],
        *[pl.col("word_count_diff").filter(pl.col("activity") == c).sum().alias(f"word_count_diff_{c}_sum") for c in activities],
        *[pl.col("word_count_diff").filter(pl.col("activity") == c).median().alias(f"word_count_diff_{c}_median") for c in activities],
        *[pl.col("word_count_diff").filter(pl.col("activity") == c).max().alias(f"word_count_diff_{c}_max") for c in activities],
    ]
    
    df = x.groupby(['id'], maintain_order=True).agg(aggs).sort("id")
    return df.to_pandas()

train_data = (pl.from_pandas(train_logs)
              .with_columns(columns)
              .with_columns(
                pl.when(pl.col("up_event") == ".")
                .then('dot')
                .when(pl.col("up_event") == "'")
                .then('abostrove')
                 .when(pl.col("up_event") == ",")
                .then('comma')
                .otherwise(pl.col("up_event")).alias("up_event")
            )
    )
train_logs_polars = time_polar_process(train_data)

test_data = (pl.from_pandas(test_logs)
              .with_columns(columns)
              .with_columns(
                pl.when(pl.col("up_event") == ".")
                .then('dot')
                .when(pl.col("up_event") == "'")
                .then('abostrove')
                 .when(pl.col("up_event") == ",")
                .then('comma')
                .otherwise(pl.col("up_event")).alias("up_event")
            )
    )

test_logs_polars = time_polar_process(test_data)

In [11]:
display(train_logs_polars.head(2))
print(train_logs_polars.shape)

,id,time_diff_q_std,time_diff_Space_std,time_diff_Backspace_std,time_diff_Shift_std,time_diff_ArrowRight_std,time_diff_Leftclick_std,time_diff_ArrowLeft_std,time_diff_dot_std,time_diff_comma_std,time_diff_ArrowDown_std,time_diff_ArrowUp_std,time_diff_Enter_std,time_diff_CapsLock_std,time_diff_abostrove_std,time_diff_Delete_std,time_diff_Unidentified_std,time_diff_q_mean,time_diff_Space_mean,time_diff_Backspace_mean,time_diff_Shift_mean,time_diff_ArrowRight_mean,time_diff_Leftclick_mean,time_diff_ArrowLeft_mean,time_diff_dot_mean,time_diff_comma_mean,time_diff_ArrowDown_mean,time_diff_ArrowUp_mean,time_diff_Enter_mean,time_diff_CapsLock_mean,time_diff_abostrove_mean,time_diff_Delete_mean,time_diff_Unidentified_mean,time_diff_q_sum,time_diff_Space_sum,time_diff_Backspace_sum,time_diff_Shift_sum,time_diff_ArrowRight_sum,time_diff_Leftclick_sum,time_diff_ArrowLeft_sum,time_diff_dot_sum,time_diff_comma_sum,time_diff_ArrowDown_sum,time_diff_ArrowUp_sum,time_diff_Enter_sum,time_diff_CapsLock_sum,time_diff_abostrove_sum,time_diff_Delete_sum,time_diff_Unidentified_sum,time_diff_q_median,time_diff_Space_median,time_diff_Backspace_median,time_diff_Shift_median,time_diff_ArrowRight_median,time_diff_Leftclick_median,time_diff_ArrowLeft_median,time_diff_dot_median,time_diff_comma_median,time_diff_ArrowDown_median,time_diff_ArrowUp_median,time_diff_Enter_median,time_diff_CapsLock_median,time_diff_abostrove_median,time_diff_Delete_median,time_diff_Unidentified_median,time_diff_q_max,time_diff_Space_max,time_diff_Backspace_max,time_diff_Shift_max,time_diff_ArrowRight_max,time_diff_Leftclick_max,time_diff_ArrowLeft_max,time_diff_dot_max,time_diff_comma_max,time_diff_ArrowDown_max,time_diff_ArrowUp_max,time_diff_Enter_max,time_diff_CapsLock_max,time_diff_abostrove_max,time_diff_Delete_max,time_diff_Unidentified_max,cursor_position_diff_q_std,cursor_position_diff_Space_std,cursor_position_diff_Backspace_std,cursor_position_diff_Shift_std,cursor_position_diff_ArrowRight_std,cursor_position_diff_Leftclick_std,cursor_position_diff_ArrowLeft_std,cursor_position_diff_dot_std,cursor_position_diff_comma_std,cursor_position_diff_ArrowDown_std,cursor_position_diff_ArrowUp_std,cursor_position_diff_Enter_std,cursor_position_diff_CapsLock_std,cursor_position_diff_abostrove_std,cursor_position_diff_Delete_std,cursor_position_diff_Unidentified_std,cursor_position_diff_q_mean,cursor_position_diff_Space_mean,cursor_position_diff_Backspace_mean,cursor_position_diff_Shift_mean,cursor_position_diff_ArrowRight_mean,cursor_position_diff_Leftclick_mean,cursor_position_diff_ArrowLeft_mean,cursor_position_diff_dot_mean,cursor_position_diff_comma_mean,cursor_position_diff_ArrowDown_mean,cursor_position_diff_ArrowUp_mean,cursor_position_diff_Enter_mean,cursor_position_diff_CapsLock_mean,cursor_position_diff_abostrove_mean,cursor_position_diff_Delete_mean,cursor_position_diff_Unidentified_mean,cursor_position_diff_q_sum,cursor_position_diff_Space_sum,cursor_position_diff_Backspace_sum,cursor_position_diff_Shift_sum,cursor_position_diff_ArrowRight_sum,cursor_position_diff_Leftclick_sum,cursor_position_diff_ArrowLeft_sum,cursor_position_diff_dot_sum,cursor_position_diff_comma_sum,cursor_position_diff_ArrowDown_sum,cursor_position_diff_ArrowUp_sum,cursor_position_diff_Enter_sum,cursor_position_diff_CapsLock_sum,cursor_position_diff_abostrove_sum,cursor_position_diff_Delete_sum,cursor_position_diff_Unidentified_sum,cursor_position_diff_q_median,cursor_position_diff_Space_median,cursor_position_diff_Backspace_median,cursor_position_diff_Shift_median,cursor_position_diff_ArrowRight_median,cursor_position_diff_Leftclick_median,cursor_position_diff_ArrowLeft_median,cursor_position_diff_dot_median,cursor_position_diff_comma_median,cursor_position_diff_ArrowDown_median,cursor_position_diff_ArrowUp_median,cursor_position_diff_Enter_median,cursor_position_diff_CapsLock_median,cursor_position_diff_abostrove_median,cursor_position_diff_Delete_median,cursor_position_diff_Unidentified_med

(2471, 316)


In [12]:
train_feats = train_feats.merge(train_logs_polars, on='id', how='inner')
test_feats = test_feats.merge(test_logs_polars, on='id', how='inner')

In [13]:
display(train_feats.head(2),train_feats.shape)

,id,event_id_max,up_time_max,action_time_max,action_time_min,action_time_mean,action_time_std,action_time_quantile,action_time_sem,action_time_sum,action_time_skew,action_time_<function NDFrame._add_numeric_operations.<locals>.kurt at 0x7826a559ad40>,activity_nunique,down_event_nunique,up_event_nunique,text_change_nunique,cursor_position_nunique,cursor_position_max,cursor_position_quantile,cursor_position_sem,cursor_position_mean,word_count_nunique,word_count_max,word_count_quantile,word_count_sem,word_count_mean,action_time_gap_1_max,action_time_gap_1_min,action_time_gap_1_mean,action_time_gap_1_std,action_time_gap_1_sum,action_time_gap_2_max,action_time_gap_2_min,action_time_gap_2_mean,action_time_gap_2_std,action_time_gap_2_sum,action_time_gap_3_max,action_time_gap_3_min,action_time_gap_3_mean,action_time_gap_3_std,action_time_gap_3_sum,action_time_gap_5_max,action_time_gap_5_min,action_time_gap_5_mean,action_time_gap_5_std,action_time_gap_5_sum,action_time_gap_10_max,action_time_gap_10_min,action_time_gap_10_mean,action_time_gap_10_std,action_time_gap_10_sum,action_time_gap_20_max,action_time_gap_20_min,action_time_gap_20_mean,action_time_gap_20_std,action_time_gap_20_sum,action_time_gap_50_max,action_time_gap_50_min,action_time_gap_50_mean,action_time_gap_50_std,action_time_gap_50_sum,action_time_gap_100_max,action_time_gap_100_min,action_time_gap_100_mean,action_time_gap_100_std,action_time_gap_100_sum,cursor_position_change_1_max,cursor_position_change_1_mean,cursor_position_change_1_std,cursor_position_change_1_sum,cursor_position_change_2_max,cursor_position_change_2_mean,cursor_position_change_2_std,cursor_position_change_2_sum,cursor_position_change_3_max,cursor_position_change_3_mean,cursor_position_change_3_std,cursor_position_change_3_sum,cursor_position_change_5_max,cursor_position_change_5_mean,cursor_position_change_5_std,cursor_position_change_5_sum,cursor_position_change_10_max,cursor_position_change_10_mean,cursor_position_change_10_std,cursor_position_change_10_sum,cursor_position_change_20_max,cursor_position_change_20_mean,cursor_position_change_20_std,cursor_position_change_20_sum,cursor_position_change_50_max,cursor_position_change_50_mean,cursor_position_change_50_std,cursor_position_change_50_sum,cursor_position_change_100_max,cursor_position_change_100_mean,cursor_position_change_100_std,cursor_position_change_100_sum,word_count_change_1_max,word_count_change_1_mean,word_count_change_1_std,word_count_change_1_sum,word_count_change_2_max,word_count_change_2_mean,word_count_change_2_std,word_count_change_2_sum,word_count_change_3_max,word_count_change_3_mean,word_count_change_3_std,word_count_change_3_sum,word_count_change_5_max,word_count_change_5_mean,word_count_change_5_std,word_count_change_5_sum,word_count_change_10_max,word_count_change_10_mean,word_count_change_10_std,word_count_change_10_sum,word_count_change_20_max,word_count_change_20_mean,word_count_change_20_std,word_count_change_20_sum,word_count_change_50_max,word_count_change_50_mean,word_count_change_50_std,word_count_change_50_sum,word_count_change_100_max,word_count_change_100_mean,word_count_change_100_std,word_count_change_100_sum,activity_0_count,activity_1_count,activity_2_count,activity_3_count,activity_4_count,down_event_0_count,down_event_1_count,down_event_2_count,down_event_3_count,down_event_4_count,down_event_5_count,down_event_6_count,down_event_7_count,down_event_8_count,down_event_9_count,down_event_10_count,down_event_11_count,down_event_12_count,down_event_13_count,down_event_14_count,down_event_15_count,up_event_0_count,up_event_1_count,up_event_2_count,up_event_3_count,up_event_4_count,up_event_5_count,up_event_6_count,up_event_7_count,up_event_8_count,up_event_9_count,up_event_10_count,up_event_11_count,up_event_12_count,up_event_13_count,up_event_14_count,up_event_15_count,text_change_0_count,text_change_1_count,text_change_2_count,text_change_3_count,text_change_4_count,text_change_5_count,text_change_6_count,tex

(2471, 506)

In [14]:
display(test_feats.head(2),test_feats.shape)

,id,event_id_max,up_time_max,action_time_max,action_time_min,action_time_mean,action_time_std,action_time_quantile,action_time_sem,action_time_sum,action_time_skew,action_time_<function NDFrame._add_numeric_operations.<locals>.kurt at 0x7826a559ad40>,activity_nunique,down_event_nunique,up_event_nunique,text_change_nunique,cursor_position_nunique,cursor_position_max,cursor_position_quantile,cursor_position_sem,cursor_position_mean,word_count_nunique,word_count_max,word_count_quantile,word_count_sem,word_count_mean,action_time_gap_1_max,action_time_gap_1_min,action_time_gap_1_mean,action_time_gap_1_std,action_time_gap_1_sum,action_time_gap_2_max,action_time_gap_2_min,action_time_gap_2_mean,action_time_gap_2_std,action_time_gap_2_sum,action_time_gap_3_max,action_time_gap_3_min,action_time_gap_3_mean,action_time_gap_3_std,action_time_gap_3_sum,action_time_gap_5_max,action_time_gap_5_min,action_time_gap_5_mean,action_time_gap_5_std,action_time_gap_5_sum,action_time_gap_10_max,action_time_gap_10_min,action_time_gap_10_mean,action_time_gap_10_std,action_time_gap_10_sum,action_time_gap_20_max,action_time_gap_20_min,action_time_gap_20_mean,action_time_gap_20_std,action_time_gap_20_sum,action_time_gap_50_max,action_time_gap_50_min,action_time_gap_50_mean,action_time_gap_50_std,action_time_gap_50_sum,action_time_gap_100_max,action_time_gap_100_min,action_time_gap_100_mean,action_time_gap_100_std,action_time_gap_100_sum,cursor_position_change_1_max,cursor_position_change_1_mean,cursor_position_change_1_std,cursor_position_change_1_sum,cursor_position_change_2_max,cursor_position_change_2_mean,cursor_position_change_2_std,cursor_position_change_2_sum,cursor_position_change_3_max,cursor_position_change_3_mean,cursor_position_change_3_std,cursor_position_change_3_sum,cursor_position_change_5_max,cursor_position_change_5_mean,cursor_position_change_5_std,cursor_position_change_5_sum,cursor_position_change_10_max,cursor_position_change_10_mean,cursor_position_change_10_std,cursor_position_change_10_sum,cursor_position_change_20_max,cursor_position_change_20_mean,cursor_position_change_20_std,cursor_position_change_20_sum,cursor_position_change_50_max,cursor_position_change_50_mean,cursor_position_change_50_std,cursor_position_change_50_sum,cursor_position_change_100_max,cursor_position_change_100_mean,cursor_position_change_100_std,cursor_position_change_100_sum,word_count_change_1_max,word_count_change_1_mean,word_count_change_1_std,word_count_change_1_sum,word_count_change_2_max,word_count_change_2_mean,word_count_change_2_std,word_count_change_2_sum,word_count_change_3_max,word_count_change_3_mean,word_count_change_3_std,word_count_change_3_sum,word_count_change_5_max,word_count_change_5_mean,word_count_change_5_std,word_count_change_5_sum,word_count_change_10_max,word_count_change_10_mean,word_count_change_10_std,word_count_change_10_sum,word_count_change_20_max,word_count_change_20_mean,word_count_change_20_std,word_count_change_20_sum,word_count_change_50_max,word_count_change_50_mean,word_count_change_50_std,word_count_change_50_sum,word_count_change_100_max,word_count_change_100_mean,word_count_change_100_std,word_count_change_100_sum,activity_0_count,activity_1_count,activity_2_count,activity_3_count,activity_4_count,down_event_0_count,down_event_1_count,down_event_2_count,down_event_3_count,down_event_4_count,down_event_5_count,down_event_6_count,down_event_7_count,down_event_8_count,down_event_9_count,down_event_10_count,down_event_11_count,down_event_12_count,down_event_13_count,down_event_14_count,down_event_15_count,up_event_0_count,up_event_1_count,up_event_2_count,up_event_3_count,up_event_4_count,up_event_5_count,up_event_6_count,up_event_7_count,up_event_8_count,up_event_9_count,up_event_10_count,up_event_11_count,up_event_12_count,up_event_13_count,up_event_14_count,up_event_15_count,text_change_0_count,text_change_1_count,text_change_2_count,text_change_3_count,text_change_4_count,text_change_5_count,text_change_6_count,tex

(3, 505)

In [15]:
# test_feats = test_feats.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
# train_feats = train_feats.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

In [16]:
# train_feats = duplicated_feature(train_feats)

In [17]:
train_feats.shape, test_feats.shape

((2471, 506), (3, 505))

# Part (2) Essay (kaggle community)

In [18]:
# Construct essays from Public Notbook
#########################
####################

class EssayConstructor:
    
    def processingInputs(self,currTextInput):
        # Where the essay content will be stored
        essayText = ""
        # Produces the essay
        for Input in currTextInput.values:
            # Input[0] = activity
            # Input[1] = cursor_position
            # Input[2] = text_change
            # Input[3] = id
            # If activity = Replace
            if Input[0] == 'Replace':
                # splits text_change at ' => '
                replaceTxt = Input[2].split(' => ')
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
                continue

            # If activity = Paste    
            if Input[0] == 'Paste':
                # DONT TOUCH
                essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
                continue

            # If activity = Remove/Cut
            if Input[0] == 'Remove/Cut':
                # DONT TOUCH
                essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
                continue

            # If activity = Move...
            if "M" in Input[0]:
                # Gets rid of the "Move from to" text
                croppedTxt = Input[0][10:]              
                # Splits cropped text by ' To '
                splitTxt = croppedTxt.split(' To ')              
                # Splits split text again by ', ' for each item
                valueArr = [item.split(', ') for item in splitTxt]              
                # Move from [2, 4] To [5, 7] = (2, 4, 5, 7)
                moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
                # Skip if someone manages to activiate this by moving to same place
                if moveData[0] != moveData[2]:
                    # Check if they move text forward in essay (they are different)
                    if moveData[0] < moveData[2]:
                        # DONT TOUCH
                        essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                    else:
                        # DONT TOUCH
                        essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
                continue                
                
            # If activity = input
            # DONT TOUCH
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
        return essayText
            
            
    def getEssays(self,df):
        # Copy required columns
        textInputDf = copy.deepcopy(df[['id', 'activity', 'cursor_position', 'text_change']])
        # Get rid of text inputs that make no change
        textInputDf = textInputDf[textInputDf.activity != 'Nonproduction']     
        # construct essay, fast 
#         tqdm.pandas()
        essay=textInputDf.groupby('id')[['activity','cursor_position', 'text_change']].apply(lambda x: self.processingInputs(x))      
        # to dataframe
        essayFrame=essay.to_frame().reset_index()
        essayFrame.columns=['id','essay']
        # Returns the essay series
        return essayFrame

In [19]:
# fast
essayConstructor = EssayConstructor()
essay = essayConstructor.getEssays(train_logs)
testessay = essayConstructor.getEssays(test_logs)

In [20]:
train_feats = train_feats.merge(essay, on='id', how='left')
test_feats = test_feats.merge(testessay, on='id', how='left')

In [21]:
train_feats.shape, test_feats.shape

((2471, 507), (3, 506))

In [22]:
# Helper functions

def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

In [23]:
AGGREGATIONS = ['count', 'mean', 'std', 'min', 'max', 'first', 'last', 'sem', q1, 'median', q3, 'skew', pd.DataFrame.kurt, 'sum']

def split_essays_into_words(df):
    essay_df = df
    essay_df['word'] = essay_df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
    essay_df = essay_df.explode('word')
    essay_df['word_len'] = essay_df['word'].apply(lambda x: len(x))
    essay_df = essay_df[essay_df['word_len'] != 0]
    return essay_df

def compute_word_aggregations(word_df):
    word_agg_df = word_df[['id','word_len']].groupby(['id']).agg(AGGREGATIONS)
    word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
    word_agg_df['id'] = word_agg_df.index
    for word_l in [2, 3, 4 , 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]:
        word_agg_df[f'word_len_ge_{word_l}_count'] = word_df[word_df['word_len'] >= word_l].groupby(['id']).count().iloc[:, 0]
        word_agg_df[f'word_len_ge_{word_l}_count'] = word_agg_df[f'word_len_ge_{word_l}_count'].fillna(0)
    word_agg_df = word_agg_df.reset_index(drop=True)
    return word_agg_df

def split_essays_into_sentences(df):
    essay_df = df
    #essay_df['id'] = essay_df.index
    essay_df['sent'] = essay_df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    essay_df = essay_df.explode('sent')
    essay_df['sent'] = essay_df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    essay_df['sent_len'] = essay_df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    essay_df['sent_word_count'] = essay_df['sent'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.sent_len!=0].reset_index(drop=True)
    return essay_df

def compute_sentence_aggregations(df):
    sent_agg_df = pd.concat(
        [df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    )
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index

    # New features intoduced here: https://www.kaggle.com/code/mcpenguin/writing-processes-to-quality-baseline-v2
    for sent_l in [15, 50, 60, 75, 80,  120, 150 , 180, 100]:
        sent_agg_df[f'sent_len_ge_{sent_l}_count'] = df[df['sent_len'] >= sent_l].groupby(['id']).count().iloc[:, 0]
        sent_agg_df[f'sent_len_ge_{sent_l}_count'] = sent_agg_df[f'sent_len_ge_{sent_l}_count'].fillna(0)
    
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    return sent_agg_df

def split_essays_into_paragraphs(df):
    essay_df = df
    #essay_df['id'] = essay_df.index
    essay_df['paragraph'] = essay_df['essay'].apply(lambda x: x.split('\n'))
    essay_df = essay_df.explode('paragraph')
    # Number of characters in paragraphs
    essay_df['paragraph_len'] = essay_df['paragraph'].apply(lambda x: len(x)) 
    # Number of words in paragraphs
    essay_df['paragraph_word_count'] = essay_df['paragraph'].apply(lambda x: len(x.split(' ')))
    essay_df = essay_df[essay_df.paragraph_len!=0].reset_index(drop=True)
    return essay_df

def compute_paragraph_aggregations(df):
    paragraph_agg_df = pd.concat(
        [df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1
    ) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df



In [24]:
# Word features for train dataset
train_word_df = split_essays_into_words(train_feats)
train_word_agg_df = compute_word_aggregations(train_word_df)

In [25]:
# display(train_word_df.head(2),
#         train_word_df.shape,
# train_word_agg_df.head(2),
#         train_word_agg_df.shape
#        )

In [26]:
# Sentence features for train dataset
train_sent_df = split_essays_into_sentences(train_feats)
train_sent_agg_df = compute_sentence_aggregations(train_sent_df)

In [27]:
# display(train_sent_df.head(2),
#         train_sent_df.shape,
# train_sent_agg_df.head(2),
#         train_sent_agg_df.shape
#        )

In [28]:
# Paragraph features for train dataset
train_paragraph_df = split_essays_into_paragraphs(train_feats)
train_paragraph_agg_df = compute_paragraph_aggregations(train_paragraph_df)

In [29]:
# Features TEST DATA
test_word_agg_df = compute_word_aggregations(split_essays_into_words(test_feats))
test_sent_agg_df = compute_sentence_aggregations(split_essays_into_sentences(test_feats))
test_paragraph_agg_df = compute_paragraph_aggregations(split_essays_into_paragraphs(test_feats))

In [30]:
train_feats = train_feats.merge(train_word_agg_df, on='id', how='left')
train_feats = train_feats.merge(train_sent_agg_df, on='id', how='left')
train_feats = train_feats.merge(train_paragraph_agg_df, on='id', how='left')

In [31]:
test_feats = test_feats.merge(test_word_agg_df, on='id', how='left')
test_feats = test_feats.merge(test_sent_agg_df, on='id', how='left')
test_feats = test_feats.merge(test_paragraph_agg_df, on='id', how='left')

In [32]:
train_feats.shape, test_feats.shape

((2471, 601), (3, 600))

In [33]:
preprocess_train = train_feats.copy()
preprocess_test = test_feats.copy()

In [34]:
del essayConstructor
del essay
del testessay
del train_word_df
del train_word_agg_df
del train_sent_df
del train_sent_agg_df
del train_paragraph_df
del train_paragraph_agg_df
del test_word_agg_df
del test_sent_agg_df
del test_paragraph_agg_df
del train_feats
del test_feats
gc.collect()

0

# Part (3) Polars Featurs (kaggle community)

In [35]:
num_cols = ['down_time', 'up_time', 'action_time', 'cursor_position', 'word_count']
activities = ['Input', 'Remove/Cut', 'Nonproduction', 'Replace', 'Paste']
events = ['q', 'Space', 'Backspace', 'Shift', 'ArrowRight', 'Leftclick', 'ArrowLeft', '.', ',', 'ArrowDown', 'ArrowUp', 'Enter', 'CapsLock', "'", 'Delete', 'Unidentified']
text_changes = ['q', ' ', '.', ',', '\n', "'", '"', '-', '?', ';', '=', '/', '\\', ':']


def count_by_values(df, colname, values):
    fts = df.select(pl.col('id').unique(maintain_order=True))
    for i, value in enumerate(values):
        tmp_df = df.group_by('id').agg(pl.col(colname).is_in([value]).sum().alias(f'{colname}_{i}_cnt'))
        fts  = fts.join(tmp_df, on='id', how='left') 
    return fts


def dev_feats(df):
    
    print("< Count by values features >")
    
    feats = count_by_values(df, 'activity', activities)
    feats = feats.join(count_by_values(df, 'text_change', text_changes), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'down_event', events), on='id', how='left') 
    feats = feats.join(count_by_values(df, 'up_event', events), on='id', how='left') 

    print("< Input words stats features >")

    temp = df.filter((~pl.col('text_change').str.contains('=>')) & (pl.col('text_change') != 'NoChange'))
    temp = temp.group_by('id').agg(pl.col('text_change').str.concat('').str.extract_all(r'q+'))
    temp = temp.with_columns(input_word_count = pl.col('text_change').list.lengths(),
                             input_word_length_mean = pl.col('text_change').apply(lambda x: np.mean([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_max = pl.col('text_change').apply(lambda x: np.max([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_std = pl.col('text_change').apply(lambda x: np.std([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_median = pl.col('text_change').apply(lambda x: np.median([len(i) for i in x] if len(x) > 0 else 0)),
                             input_word_length_skew = pl.col('text_change').apply(lambda x: skew([len(i) for i in x] if len(x) > 0 else 0)))
    temp = temp.drop('text_change')
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Numerical columns features >")

    temp = df.group_by("id").agg(pl.sum('action_time').suffix('_sum'), pl.mean(num_cols).suffix('_mean'), pl.std(num_cols).suffix('_std'),
                                 pl.median(num_cols).suffix('_median'), pl.min(num_cols).suffix('_min'), pl.max(num_cols).suffix('_max'),
                                 pl.quantile(num_cols, 0.5).suffix('_quantile'))
    feats = feats.join(temp, on='id', how='left') 


    print("< Categorical columns features >")
    
    temp  = df.group_by("id").agg(pl.n_unique(['activity', 'down_event', 'up_event', 'text_change']))
    feats = feats.join(temp, on='id', how='left') 


    
    print("< Idle time features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.group_by("id").agg(inter_key_largest_lantency = pl.max('time_diff'),
                                   inter_key_median_lantency = pl.median('time_diff'),
                                   mean_pause_time = pl.mean('time_diff'),
                                   std_pause_time = pl.std('time_diff'),
                                   total_pause_time = pl.sum('time_diff'),
                                   pauses_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 0.5) & (pl.col('time_diff') < 1)).count(),
                                   pauses_1_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1) & (pl.col('time_diff') < 1.5)).count(),
                                   pauses_1_half_sec = pl.col('time_diff').filter((pl.col('time_diff') > 1.5) & (pl.col('time_diff') < 2)).count(),
                                   pauses_2_sec = pl.col('time_diff').filter((pl.col('time_diff') > 2) & (pl.col('time_diff') < 3)).count(),
                                   pauses_3_sec = pl.col('time_diff').filter(pl.col('time_diff') > 3).count(),)
    feats = feats.join(temp, on='id', how='left') 
    
    print("< P-bursts features >")

    temp = df.with_columns(pl.col('up_time').shift().over('id').alias('up_time_lagged'))
    temp = temp.with_columns((abs(pl.col('down_time') - pl.col('up_time_lagged')) / 1000).fill_null(0).alias('time_diff'))
    temp = temp.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('time_diff')<2)
    temp = temp.with_columns(pl.when(pl.col("time_diff") & pl.col("time_diff").is_last()).then(pl.count()).over(pl.col("time_diff").rle_id()).alias('P-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('P-bursts').suffix('_mean'), pl.std('P-bursts').suffix('_std'), pl.count('P-bursts').suffix('_count'),
                                   pl.median('P-bursts').suffix('_median'), pl.max('P-bursts').suffix('_max'),
                                   pl.first('P-bursts').suffix('_first'), pl.last('P-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left') 


    print("< R-bursts features >")

    temp = df.filter(pl.col('activity').is_in(['Input', 'Remove/Cut']))
    temp = temp.with_columns(pl.col('activity').is_in(['Remove/Cut']))
    temp = temp.with_columns(pl.when(pl.col("activity") & pl.col("activity").is_last()).then(pl.count()).over(pl.col("activity").rle_id()).alias('R-bursts'))
    temp = temp.drop_nulls()
    temp = temp.group_by("id").agg(pl.mean('R-bursts').suffix('_mean'), pl.std('R-bursts').suffix('_std'), 
                                   pl.median('R-bursts').suffix('_median'), pl.max('R-bursts').suffix('_max'),
                                   pl.first('R-bursts').suffix('_first'), pl.last('R-bursts').suffix('_last'))
    feats = feats.join(temp, on='id', how='left')
    
    return feats


def train_valid_split(data_x, data_y, train_idx, valid_idx):
    x_train = data_x.iloc[train_idx]
    y_train = data_y[train_idx]
    x_valid = data_x.iloc[valid_idx]
    y_valid = data_y[valid_idx]
    return x_train, y_train, x_valid, y_valid


def evaluate(data_x, data_y, model, random_state=42, n_splits=5, test_x=None):
    skf    = StratifiedKFold(n_splits=n_splits, random_state=random_state, shuffle=True)
    test_y = np.zeros(len(data_x)) if (test_x is None) else np.zeros((len(test_x), n_splits))
    for i, (train_index, valid_index) in enumerate(skf.split(data_x, data_y.astype(str))):
        train_x, train_y, valid_x, valid_y = train_valid_split(data_x, data_y, train_index, valid_index)
        model.fit(train_x, train_y)
        if test_x is None:
            test_y[valid_index] = model.predict(valid_x)
        else:
            test_y[:, i] = model.predict(test_x)
    return test_y if (test_x is None) else np.mean(test_y, axis=1)
def q1(x):
    return x.quantile(0.25)
def q3(x):
    return x.quantile(0.75)

AGGREGATIONS = ['count', 'mean', 'min', 'max', 'first', 'last', q1, 'median', q3, 'sum', 'std', 'sem']

def reconstruct_essay(currTextInput):
    essayText = ""
    for Input in currTextInput.values:
        if Input[0] == 'Replace':
            replaceTxt = Input[2].split(' => ')
            essayText = essayText[:Input[1] - len(replaceTxt[1])] + replaceTxt[1] + essayText[Input[1] - len(replaceTxt[1]) + len(replaceTxt[0]):]
            continue
        if Input[0] == 'Paste':
            essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
            continue
        if Input[0] == 'Remove/Cut':
            essayText = essayText[:Input[1]] + essayText[Input[1] + len(Input[2]):]
            continue
        if "M" in Input[0]:
            croppedTxt = Input[0][10:]
            splitTxt = croppedTxt.split(' To ')
            valueArr = [item.split(', ') for item in splitTxt]
            moveData = (int(valueArr[0][0][1:]), int(valueArr[0][1][:-1]), int(valueArr[1][0][1:]), int(valueArr[1][1][:-1]))
            if moveData[0] != moveData[2]:
                if moveData[0] < moveData[2]:
                    essayText = essayText[:moveData[0]] + essayText[moveData[1]:moveData[3]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[3]:]
                else:
                    essayText = essayText[:moveData[2]] + essayText[moveData[0]:moveData[1]] + essayText[moveData[2]:moveData[0]] + essayText[moveData[1]:]
            continue
        essayText = essayText[:Input[1] - len(Input[2])] + Input[2] + essayText[Input[1] - len(Input[2]):]
    return essayText


def get_essay_df(df):
    df       = df[df.activity != 'Nonproduction']
    temp     = df.groupby('id').apply(lambda x: reconstruct_essay(x[['activity', 'cursor_position', 'text_change']]))
    essay_df = pd.DataFrame({'id': df['id'].unique().tolist()})
    essay_df = essay_df.merge(temp.rename('essay'), on='id')
    return essay_df


def word_feats(df):
    essay_df = df
    df['word'] = df['essay'].apply(lambda x: re.split(' |\\n|\\.|\\?|\\!',x))
    df = df.explode('word')
    df['word_len'] = df['word'].apply(lambda x: len(x))
    df = df[df['word_len'] != 0]

    word_agg_df = df[['id','word_len']].groupby(['id']).agg(AGGREGATIONS)
    word_agg_df.columns = ['_'.join(x) for x in word_agg_df.columns]
    word_agg_df['id'] = word_agg_df.index
    word_agg_df = word_agg_df.reset_index(drop=True)
    return word_agg_df


def sent_feats(df):
    df['sent'] = df['essay'].apply(lambda x: re.split('\\.|\\?|\\!',x))
    df = df.explode('sent')
    df['sent'] = df['sent'].apply(lambda x: x.replace('\n','').strip())
    # Number of characters in sentences
    df['sent_len'] = df['sent'].apply(lambda x: len(x))
    # Number of words in sentences
    df['sent_word_count'] = df['sent'].apply(lambda x: len(x.split(' ')))
    df = df[df.sent_len!=0].reset_index(drop=True)

    sent_agg_df = pd.concat([df[['id','sent_len']].groupby(['id']).agg(AGGREGATIONS), 
                             df[['id','sent_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1)
    sent_agg_df.columns = ['_'.join(x) for x in sent_agg_df.columns]
    sent_agg_df['id'] = sent_agg_df.index
    sent_agg_df = sent_agg_df.reset_index(drop=True)
    sent_agg_df.drop(columns=["sent_word_count_count"], inplace=True)
    sent_agg_df = sent_agg_df.rename(columns={"sent_len_count":"sent_count"})
    return sent_agg_df


def parag_feats(df):
    df['paragraph'] = df['essay'].apply(lambda x: x.split('\n'))
    df = df.explode('paragraph')
    # Number of characters in paragraphs
    df['paragraph_len'] = df['paragraph'].apply(lambda x: len(x)) 
    # Number of words in paragraphs
    df['paragraph_word_count'] = df['paragraph'].apply(lambda x: len(x.split(' ')))
    df = df[df.paragraph_len!=0].reset_index(drop=True)
    
    paragraph_agg_df = pd.concat([df[['id','paragraph_len']].groupby(['id']).agg(AGGREGATIONS), 
                                  df[['id','paragraph_word_count']].groupby(['id']).agg(AGGREGATIONS)], axis=1) 
    paragraph_agg_df.columns = ['_'.join(x) for x in paragraph_agg_df.columns]
    paragraph_agg_df['id'] = paragraph_agg_df.index
    paragraph_agg_df = paragraph_agg_df.reset_index(drop=True)
    paragraph_agg_df.drop(columns=["paragraph_word_count_count"], inplace=True)
    paragraph_agg_df = paragraph_agg_df.rename(columns={"paragraph_len_count":"paragraph_count"})
    return paragraph_agg_df

def product_to_keys(logs, essays):
    essays['product_len'] = essays.essay.str.len()
    tmp_df = logs[logs.activity.isin(['Input', 'Remove/Cut'])].groupby(['id']).agg({'activity': 'count'}).reset_index().rename(columns={'activity': 'keys_pressed'})
    essays = essays.merge(tmp_df, on='id', how='left')
    essays['product_to_keys'] = essays['product_len'] / essays['keys_pressed']
    return essays[['id', 'product_to_keys']]

def get_keys_pressed_per_second(logs):
    temp_df = logs[logs['activity'].isin(['Input', 'Remove/Cut'])].groupby(['id']).agg(keys_pressed=('event_id', 'count')).reset_index()
    temp_df_2 = logs.groupby(['id']).agg(min_down_time=('down_time', 'min'), max_up_time=('up_time', 'max')).reset_index()
    temp_df = temp_df.merge(temp_df_2, on='id', how='left')
    temp_df['keys_per_second'] = temp_df['keys_pressed'] / ((temp_df['max_up_time'] - temp_df['min_down_time']) / 1000)
    return temp_df[['id', 'keys_per_second']]


In [36]:
data_path     = '/kaggle/input/linking-writing-processes-to-writing-quality/'
train_logs    = pl.scan_csv(data_path + 'train_logs.csv')#[:200]
train_feats   = dev_feats(train_logs)
train_feats   = train_feats.collect().to_pandas()

print('< Essay Reconstruction >')
train_logs             = train_logs.collect().to_pandas()
train_essays           = get_essay_df(train_logs)
train_feats            = train_feats.merge(word_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(sent_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(parag_feats(train_essays), on='id', how='left')
train_feats            = train_feats.merge(get_keys_pressed_per_second(train_logs), on='id', how='left')
train_feats            = train_feats.merge(product_to_keys(train_logs, train_essays), on='id', how='left')


print('< Mapping >')
train_scores   = pd.read_csv(data_path + 'train_scores.csv')
data           = train_feats.merge(train_scores, on='id', how='left')
x              = data.drop(['id', 'score'], axis=1)
y              = data['score'].values
print(f'Number of features: {len(x.columns)}')


print('< Testing Data >')
test_logs   = pl.scan_csv(data_path + 'test_logs.csv')
test_feats  = dev_feats(test_logs)
test_feats  = test_feats.collect().to_pandas()

test_logs             = test_logs.collect().to_pandas()
test_essays           = get_essay_df(test_logs)
test_feats            = test_feats.merge(word_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(sent_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(parag_feats(test_essays), on='id', how='left')
test_feats            = test_feats.merge(get_keys_pressed_per_second(test_logs), on='id', how='left')
test_feats            = test_feats.merge(product_to_keys(test_logs, test_essays), on='id', how='left')

# test_feats.to_csv('silver_bullet_feature_test.csv',index=False)

< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >
< Essay Reconstruction >
< Mapping >
Number of features: 175
< Testing Data >
< Count by values features >
< Input words stats features >
< Numerical columns features >
< Categorical columns features >
< Idle time features >
< P-bursts features >
< R-bursts features >


In [37]:
silver_bullet_train = train_feats.copy()
silver_bullet_test = test_feats.copy()
# train_feats.to_csv('part_3_training.csv',index=False)

In [38]:
intesect_feat = np.intersect1d(silver_bullet_train.columns, preprocess_train.columns)
# ID = silver_bullet_train.id
# silver_bullet_train.drop(intesect_feat,axis=1,inplace=True)
# silver_bullet_train['id'] = ID
drop_col = [col for col in intesect_feat if col not in ['id']]
silver_bullet_train.drop(drop_col, axis=1, inplace=True)
silver_bullet_test.drop(drop_col, axis=1, inplace=True)

In [39]:
train_feats = silver_bullet_train.merge(preprocess_train, on='id')
test_feats = silver_bullet_test.merge(preprocess_test, on='id')

In [40]:
train_feats.shape, test_feats.shape

((2471, 701), (3, 700))

In [41]:
# train_feats.to_csv('part_4_training.csv',index=False)
# train_feats = pd.read_csv('part_4_training.csv')
# test_feats = pd.read_csv('part_4_training.csv').head(3)

# Part 4 Advance Feature ( Articals and Papers)

### 4.1 Features related to timings of pauses.

A majority of features we extracted
are related to the timings of pauses. Most features are related to the IKI: the
time from a key press until the next key press.

In [42]:
def paper_features(df):
    
    # IKI
    df['time_diff'] = ((df['down_time'] - df.groupby('id')['up_time'].diff()) * 0.001).fillna(0)

    initial_pause_time = df.groupby('id')['time_diff'].min()
    total_time = df.groupby('id')['time_diff'].max()
    
    
    # Group data by 'ID' and calculate IKI metrics
    iki_metrics = df.groupby('id')['time_diff'].apply(lambda x: x.diff().dropna())
    metrics = iki_metrics.groupby(level=0).agg(['mean', 'median', 'std', 'max', 'sum','skew'])
    
    # Group data by 'ID' and calculate IKI letters words
    df['prev_timestamp'] = df.groupby('id')['time_diff'].shift(-1)
    
    df['IKI_between_words'] = df['time_diff'] - df['prev_timestamp']
    
    IKI_between_letters_metrics = df.groupby('id')['IKI_between_words'].agg(['mean', 'std', 'sum','max','min','skew'])

    
    return initial_pause_time, total_time , metrics, IKI_between_letters_metrics


In [43]:
pass
'''
• Initial pause time. Time from the start of the keystroke logging (assignment
start) until first key press .

• Total time. Time from start of the keystroke logging (assignment start) to last
key release 
• Mean, Median, SD, and maximum IKI. Metrics of the time from a key press
until the next key press.

• Mean and SD IKI within word. Metrics of all IKI of keystrokes within
words 
• Mean and SD IKI between words. Metrics of all IKI of keystrokes between
words, i.e., the last letter of a word and the space between words.

• Mean and SD time between words. Metrics of the time from key press of the
last letter of a word until the key press of the first letter of the next word.

• Mean and SD time between sentences. Metrics of the time from key press of
the end of a sentence marker until the key press of the first letter of the next sentence.

'''
pass

###  4.2 Mean and SD.. etc IKI within letters ( one press)

In [44]:
'''
Metrics of the time from a key press
until the next key press
'''

for df, df_logs in [(train_feats, train_logs), (test_feats, test_logs)]:
    
    paper_feature = paper_features(df_logs)
    
    
    # Initial pause time (min)
    df['initial_pause_time'] = df['id'].map(paper_feature[0])
    
    # Total time
    df['total_time']= df['id'].map(paper_feature[1])
    
    # Mean, Median SD.. etc IKI 
    df['IKI_mean'] = df['id'].map(paper_feature[2]['mean'])    
    df['IKI_median'] = df['id'].map(paper_feature[2]['median'])    
    df['IKI_max'] = df['id'].map(paper_feature[2]['max'])    
    df['IKI_std'] = df['id'].map(paper_feature[2]['std'])    
    df['IKI_sum'] = df['id'].map(paper_feature[2]['sum'])    
    df['IKI_skew'] = df['id'].map(paper_feature[2]['skew'])
    df['IKI_bw_letter_mean'] = df['id'].map(paper_feature[3]['mean'])    
    df['IKI_bw_letter_std'] = df['id'].map(paper_feature[3]['std'])    
    df['IKI_bw_letter_sum'] = df['id'].map(paper_feature[3]['sum'])    
    df['IKI_bw_letter_max'] = df['id'].map(paper_feature[3]['max'])    
    df['IKI_bw_letter_min'] = df['id'].map(paper_feature[3]['min'])    
    df['IKI_bw_letter_skew'] = df['id'].map(paper_feature[3]['skew'])

### 4.3 Time and  IKI between words & Sentences

In [45]:


'''

• Mean and SD IKI between Sentences

• Mean and SD IKI between words. Metrics of all IKI of keystrokes between
words, i.e., the last letter of a word and the space between words.

• Mean and SD time between words. Metrics of the time from key press of the
last letter of a word until the key press of the first letter of the next word.

• Mean and SD time between sentences. Metrics of the time from key press of
the end of a sentence marker until the key press of the first letter of the next sentence.

'''

for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):

    # Shift 'timestamp' column by one row within each 'ID' group to get the next sentence's timestamp
    df_logs['next_timestamp'] = df_logs.groupby('id')['time_diff'].shift(-1)

    # Calculate time for each ID
    df_logs['time_between_sentences'] = df_logs['next_timestamp'] - df_logs['time_diff']


    ########## Mean and SD IKI between Sentences#########################

    df_logs['Sentences_End'] = (df_logs['text_change'].isin(['.', '?', '!']) & (df_logs.activity =='Input')).astype(int)


    sentences_logs = df_logs[df_logs['Sentences_End'] == 1]  # Consider only sentence end markers

    # Remove NaN values resulting from the shift operation for the last sentence in each ID
    sentences_logs = sentences_logs.dropna(subset=['time_between_sentences'])

    # Group data by 'ID' and calculate Mean and SD time between sentences
    time_between_sentences_metrics = sentences_logs.groupby('id')['time_between_sentences'].agg(['mean', 'std','sum','max'])

    IKI_between_sentences_metrics = sentences_logs.groupby('id')['action_time'].agg(['mean', 'std','sum','max'])


    ########## Mean and SD IKI between Words#########################

    df_logs['word_End'] = (df_logs['text_change'].isin(['.', '?', '!', ' ']) & (df_logs.activity =='Input')).astype(int)

    word_logs = df_logs[df_logs['word_End'] == 1]  # Consider only words end

    # Remove NaN values resulting from the shift operation for the last sentence in each ID
    word_logs = word_logs.dropna(subset=['time_between_sentences'])

    # Group data by 'ID' and calculate Mean and SD time between word
    time_between_word_metrics = word_logs.groupby('id')['time_between_sentences'].agg(['mean', 'std','sum','max'])
    
    # Group data by 'ID' and calculate Mean and SD IKI between word
    IKI_between_word_metrics = word_logs.groupby('id')['action_time'].agg(['mean', 'std','sum','max'])
    
    
    time_between_sentences_metrics.columns = [f'time_between_sentences_{x}' for x in time_between_sentences_metrics.columns]
    time_between_word_metrics.columns = [f'time_between_words_{x}' for x in time_between_word_metrics.columns]


    IKI_between_sentences_metrics.columns = [ f'IKI_between_sentences_metrics_{x}' for x in IKI_between_sentences_metrics.columns]
    IKI_between_word_metrics.columns = [ f'IKI_between_word_metrics_{x}' for x in IKI_between_word_metrics.columns]


    df = df.merge(time_between_sentences_metrics, on='id', how='left')
    df = df.merge(time_between_word_metrics, on='id', how='left')

    df = df.merge(IKI_between_sentences_metrics, on='id', how='left')
    df = df.merge(IKI_between_word_metrics, on='id', how='left')

  0%|          | 0/2 [00:00<?, ?it/s]

### 4.4 Number of IKI of specific length.

In [46]:
'''
ten features were extracted: the number
from IKI between 0.5–1.0 s, to 9.0- 10.0
'''
# Calculate Inter-Keystroke Intervals (IKIs) for each ID
train_logs['IKI'] = train_logs.groupby('id')['time_diff'].diff()
test_logs['IKI'] = test_logs.groupby('id')['time_diff'].diff()

for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):

    # Define time ranges for IKIs
    time_ranges = [(0.5, 1.0), (1.0, 1.5), (1.5, 2.0), (2.0, 3.0), (3.0, 4.0),(4.0, 5.0),  (5.0, 6.0), (6.0, 7.0), (7.0, 8.0),  (8.0, 9.0),  (9.0, 10.0), (3.0, float('inf') ), (4.0, float('inf') ), (5.0, float('inf') ), (6.0, float('inf') ), (7.0, float('inf'))]  # in seconds

    # Initialize counters for each time range
    iki_counts = {f'IKI_{start}-{end}': 0 for start, end in time_ranges}

    IKI_RANG_FEATURES = []
    # Count IKIs falling within specified time ranges
    for start, end in tqdm(time_ranges):

        range_name = f'IKI_{start}-{end}'

        df_logs[range_name] = ((df_logs['IKI'] >= start) & (df_logs['IKI'] < end)).astype(int)

        df[range_name + '_sum'] = df_logs['id'].map(df_logs.groupby('id')[range_name].sum())

        IKI_RANG_FEATURES.append(range_name + '_sum')

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

  0%|          | 0/16 [00:00<?, ?it/s]

### 4.5 Percentage long pauses between words

In [47]:
'''
Percentage long pauses between words. Number of pauses between words
longer than two SD from the mean IKI within the copy task, divided by the total
number of pauses between words
'''
for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):
    
    df_logs['IKI_between_word_mean'] = df_logs.groupby('id')['time_between_sentences'].transform('mean')

    df_logs['IKI_between_word_std'] = df_logs.groupby('id')['time_between_sentences'].transform('std')

    word_logs_longs_than_2std_pause = (df_logs[(df_logs['word_End'] == 1) & (df_logs['time_between_sentences'] > (df_logs['IKI_between_word_mean'] + (2 * df_logs['IKI_between_word_std'] )))]).groupby('id').size()  # Consider only words end

    df['Total_Number_of_pause'] = df['id'].map(word_logs.groupby('id').size())

    df['Number_of_Pause_bw_word'] = df['id'].map(word_logs_longs_than_2std_pause).fillna(0)

    df['Percentage_long_pauses_between_words'] = df['Number_of_Pause_bw_word'] / df['Total_Number_of_pause'] * 100

  0%|          | 0/2 [00:00<?, ?it/s]

### 4.6 Features related to Revisions.

features related to revisions were extracted.
Again, all time-based features were log transformed and all values above the 95th
percentile were removed.

In [48]:
# Note. the Number of Revisions. feaures was not calculated
# Note. the Number of leading-edge revisions. feaures was not calculated

In [49]:

for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):
    df_logs['IKI'] = df_logs.groupby('id')['time_diff'].diff()
    
    # Number of backspaces. Number of backspaces and delete keystrokes
    df['Number_of_backspaces'] = df['id'].map(df_logs.loc[(df_logs.up_event =='Backspace') & (df_logs.activity =='Remove/Cut')].groupby('id')['id'].count()).fillna(0)

    # Mean and SD time in multiple backspacing.
    df['backspacing_time_mean'] = df['id'].map(df_logs.loc[(df_logs.up_event =='Backspace') & (df_logs.activity =='Remove/Cut')].groupby('id')['IKI'].mean())
    df['backspacing_time_std'] = df['id'].map(df_logs.loc[(df_logs.up_event =='Backspace') & (df_logs.activity =='Remove/Cut')].groupby('id')['IKI'].std())
    
    df['backspacing_time_max'] = df['id'].map(df_logs.loc[(df_logs.up_event =='Backspace') & (df_logs.activity =='Remove/Cut')].groupby('id')['IKI'].max())
    df['backspacing_time_sum'] = df['id'].map(df_logs.loc[(df_logs.up_event =='Backspace') & (df_logs.activity =='Remove/Cut')].groupby('id')['IKI'].sum())
    df['backspacing_time_skew'] = df['id'].map(df_logs.loc[(df_logs.up_event =='Backspace') & (df_logs.activity =='Remove/Cut')].groupby('id')['IKI'].skew())
    df['backspacing_time_count'] = df['id'].map(df_logs.loc[(df_logs.up_event =='Backspace') & (df_logs.activity =='Remove/Cut')].groupby('id')['IKI'].count())

  0%|          | 0/2 [00:00<?, ?it/s]

In [50]:
# IKI_between_words_metrics_mean

for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):
    df_logs['IKI'] = df_logs.groupby('id')['time_diff'].diff()
    
    df_logs['IKI_mean'] = df_logs.groupby('id')['time_diff'].transform('mean')

    # IKI_between_words_metrics_std
    df_logs['IKI_std'] = df_logs.groupby('id')['time_diff'].transform('std')

    # Calculate long pauses (longer than 2 SD from mean IKI)
    df['long_pauses_count'] = df['id'].map(df_logs[df_logs['time_diff'] > (df_logs['IKI_mean'] + (2 * df_logs['IKI_std'] ))].groupby('id')['IKI'].count())
    df['long_pauses_sum'] = df['id'].map(df_logs[df_logs['time_diff'] > (df_logs['IKI_mean'] + (2 * df_logs['IKI_std'] ))].groupby('id')['IKI'].sum())
    df['long_pauses_std'] = df['id'].map(df_logs[df_logs['time_diff'] > (df_logs['IKI_mean'] + (2 * df_logs['IKI_std'] ))].groupby('id')['IKI'].std())
    df['long_pauses_mean'] = df['id'].map(df_logs[df_logs['time_diff'] > (df_logs['IKI_mean'] + (2 * df_logs['IKI_std'] ))].groupby('id')['IKI'].mean())


    df['percentage_long_pauses'] = df['long_pauses_count'] / df['event_id_max'] * 100

  0%|          | 0/2 [00:00<?, ?it/s]

In [51]:
for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):
    
    df_logs['IKI_mean'] = df_logs.groupby('id')['time_diff'].transform('mean')
    # Number of revisions
        
    df['Number of Revesions'] = df['id'].map((df_logs.loc[(df_logs.activity == 'Paste') | (df_logs.activity == 'Replace') | (df_logs.activity == 'Remove/Cut')].groupby('id').size()).fillna(0))
    # Number of leading-edge revisions
    
    for num in tqdm([2, 3, 4, 5, 10, 14, 50]):
        
        df[f'Number of revisions Backspace {num} word'] = df['id'].map((df_logs.loc[(df_logs.activity == 'Remove/Cut') & (df_logs['up_event']=='Backspace') & (df_logs.word_count == num)].groupby('id').size()).fillna(0))

        df[f'Number of revisions Remove/Cut {num} word'] = df['id'].map((df_logs.loc[(df_logs.activity == 'Remove/Cut') & (df_logs.word_count == num)].groupby('id').size()).fillna(0))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

  0%|          | 0/7 [00:00<?, ?it/s]

In [52]:
# the number of leading-edge revisions for each user
for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):
    
    df_logs['IKI'] = df_logs.groupby('id')['time_diff'].diff()
    
    df['leading-edge revisions_max'] = df['id'].map(df_logs[(df_logs.activity=='Input') &( df_logs['time_diff'] <=(2* df_logs['IKI_std']))].groupby('id')['IKI'].max())
    
    df['leading-edge revisions_sum'] = df['id'].map(df_logs[(df_logs.activity=='Input') &( df_logs['time_diff'] <=(2* df_logs['IKI_std']))].groupby('id')['IKI'].sum())
    
    df['leading-edge revisions_std'] = df['id'].map(df_logs[(df_logs.activity=='Input') &( df_logs['time_diff'] <=(2* df_logs['IKI_std']))].groupby('id')['IKI'].std())
    
    df['leading-edge revisions_mean'] = df['id'].map(df_logs[(df_logs.activity=='Input') &( df_logs['time_diff'] <=(2* df_logs['IKI_std']))].groupby('id')['IKI'].mean())
    
    df['leading-edge revisions_count'] = df['id'].map(df_logs[(df_logs.activity=='Input') &( df_logs['time_diff'] <=(2* df_logs['IKI_std']))].groupby('id')['IKI'].count())
    

  0%|          | 0/2 [00:00<?, ?it/s]

In [53]:
# Calculate the time duration for single backspacing sequences
for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):

    single_backspace_duration = []
    multiple_backspace_duration = []
    
    single_backspace_duration_mean = dict()
    single_backspace_duration_std = dict()
    single_backspace_duration_sum = dict()
    single_backspace_duration_min = dict()
    single_backspace_duration_max = dict()
    
    multiple_backspace_duration_mean = dict()
    multiple_backspace_duration_std = dict()
    multiple_backspace_duration_sum = dict()
    multiple_backspace_duration_min = dict()
    multiple_backspace_duration_max = dict()
    
    for user, group in tqdm(df_logs.groupby('id')):

        backspace_times = group[group['up_event']=='Backspace']['time_diff'].values

        if len(backspace_times) > 1:

            single_backspace_dur = backspace_times[1:] - backspace_times[:-1]
            single_backspace_duration.extend(single_backspace_dur)
            
            single_backspace_duration_mean[user] = single_backspace_dur.mean()
            single_backspace_duration_std[user] = single_backspace_dur.std()
            single_backspace_duration_sum[user] = single_backspace_dur.sum()
            single_backspace_duration_min[user] = single_backspace_dur.min()
            single_backspace_duration_max[user] = single_backspace_dur.max()
            
        if len(backspace_times) > 2:

            multiple_backspace_dur = backspace_times[1:] - backspace_times[:-1]
            multiple_backspace_duration.extend(multiple_backspace_dur)
            
            multiple_backspace_duration_mean[user] = multiple_backspace_dur.mean()
            multiple_backspace_duration_std[user] = multiple_backspace_dur.std()
            multiple_backspace_duration_sum[user] = multiple_backspace_dur.sum()
            multiple_backspace_duration_min[user] = multiple_backspace_dur.min()
            multiple_backspace_duration_max[user] = multiple_backspace_dur.max()

    df['sequence_of_backspaces_mean'] = df['id'].map(single_backspace_duration_mean)
    df['sequence_of_backspaces_std'] = df['id'].map(single_backspace_duration_std)
    df['sequence_of_backspaces_sum'] = df['id'].map(single_backspace_duration_sum)
    df['sequence_of_backspaces_max'] = df['id'].map(single_backspace_duration_max)
    df['sequence_of_backspaces_min'] = df['id'].map(single_backspace_duration_min)
    
    df['multiple_of_backspaces_mean'] = df['id'].map(multiple_backspace_duration_mean)
    df['multiple_of_backspaces_std'] = df['id'].map(multiple_backspace_duration_std)
    df['multiple_of_backspaces_sum'] = df['id'].map(multiple_backspace_duration_sum)
    df['multiple_of_backspaces_max'] = df['id'].map(multiple_backspace_duration_max)
    df['multiple_of_backspaces_min'] = df['id'].map(multiple_backspace_duration_min)

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/2471 [00:00<?, ?it/s]

  0%|          | 0/3 [00:00<?, ?it/s]

In [54]:
# percentage of characters in the final text based on the total number of keystrokes
for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):
    
    df['number_of_characters'] = df['id'].map((df_logs.loc[(df_logs['activity'] == 'Input')  & (df_logs['text_change'] =='q')].groupby('id').size()).fillna(0))
    df['number_of_Kaystros'] = df['id'].map(df_logs.groupby('id').size())
    
    df['number_of_characters_at_leading_edg'] = df['id'].map((df_logs.loc[(df_logs['activity'] == 'Input')  & (df_logs['text_change'] =='q')  & ( df_logs['time_diff'] <=(2* df_logs['IKI_std']))].groupby('id').size()).fillna(0))
    
    
    df['percentage_final_text'] = df['number_of_Kaystros'] / df['number_of_Kaystros'] * 100
    df['Percentage_haracters_at_leading_edge'] = df['number_of_Kaystros'] / df['number_of_Kaystros'] * 100

    

  0%|          | 0/2 [00:00<?, ?it/s]

### 4.7  Features related to fluency 

In [55]:
'''
Fluency in writing has been argued to be reflected
in verbosity (production), process variance, revision, and pausing behavior (Van
Waes & Leijten, 2015). Here, we solely focus on the burstiness of the writing.
Sentences are argued to be composed in sentence parts, also known as written
language bursts (Kaufer et al., 1986). Written language bursts, hereafter ‘bursts’,
are defined as sequences of text production without a long pause and without afocus 
shift (e.g., to the task or an online dictionary). To account for individual
differences in typing speed, all bursts are defined as sequences of keystrokes that
do not contain pauses longer than two SD from the mean IKI within the copy task
of the same participant
'''


for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):

    # Calculate IKI (Inter-Keystroke Intervals)
    df_logs['IKI'] = df_logs.groupby('id')['time_diff'].diff()

    # Caclulate bursts based on the defined criteria
    df_logs['Burst'] = (df_logs['IKI'] <= 2 * df_logs.groupby('id')['IKI'].transform('std')) | df_logs['IKI'].isnull()

    # Identify bursts by consecutive burst indicators
    df_logs['Burst_ID'] = (df_logs['Burst'] != df_logs['Burst'].shift()).cumsum()

    
    '''Number of bursts'''
    # Add Feature that Calculate the number of bursts for each user
    df['num_bursts'] = df['id'].map(df_logs.groupby('id')['Burst_ID'].nunique())
    
    
    
    '''Mean, SD, and maximum..etc number of characters per burst'''
    # Calculate the maximum number of characters per burst
    df['max_chars_per_burst'] = df['id'].map(df_logs.groupby(['id', 'Burst_ID']).size().groupby('id').max())

    # Calculate the maximum number of characters per burst
    df['mean_chars_per_burst'] = df['id'].map(df_logs.groupby(['id', 'Burst_ID']).size().groupby('id').mean())
    
    # Calculate the standard deviation of characters per burst
    df['std_chars_per_burst'] = df['id'].map(df_logs.groupby(['id', 'Burst_ID']).size().groupby('id').std())
    
    # Calculate the sum of characters per burst
    df['sum_chars_per_burst'] = df['id'].map(df_logs.groupby(['id', 'Burst_ID']).size().groupby('id').sum())

    
    '''Percentage of R-bursts. Number of revision bursts at the leading edge ending
        in a revision, divided by the total number of bursts'''
    
     # Calculate the number of revision bursts at the leading edge ending in a revision
    df['num_revision_bursts'] =  df['id'].map(df_logs[(df_logs['up_event'] == 'Backspace') & (df_logs['Burst'] == True)].groupby('id')['Burst_ID'].nunique())
    
    # Calculate Percentage of R-bursts
    df['Percentage_of_R_bursts'] = df['num_revision_bursts'] / df['num_bursts'] * 100
    
    
    '''Percentage of I-bursts. Number of insertion bursts produced away from the
        leading edge, divided by the total number of bursts'''
    
    # Calculate Number of insertion bursts produced away from the leading edge
    df['num_insertion_bursts'] = df['id'].map(df_logs[(df_logs['activity'] == 'Input') & (df_logs['Burst'] == True)].groupby('id')['Burst_ID'].nunique())
    
    # Calculate Percentage of I-bursts
    df['Percentage_of_I_bursts'] = df['num_insertion_bursts'] / df['num_bursts'] * 100

    

  0%|          | 0/2 [00:00<?, ?it/s]

In [56]:
pass
'''
• Percentage of words in P-bursts. Number of words in ’clean’ production
bursts both initiated and terminated by a long pause (not a revision), divided
by the total number of words (Baaijen et al., 2012).
• Number of production cycles. Number of groups of bursts without interruptions
by other events (i.e., all events not resulting in a character being typed,
see also features related to other events), divided by the number of words
(Baaijen & Galbraith, 2018).
• Percentage of linear transitions between words. Number of times the transition
to the next word was not interrupted by other events, divided by the total
number of transitions between words (Baaijen & Galbraith, 2018).
• Percentage of linear transitions between sentences. Number of times the
transition to the next sentence was not interrupted by other events, divided by
the total number of transitions between sentences (Baaijen & Galbraith, 2018).
'''
pass

### 4.8 Features related to verbosity

In [57]:
# First caclulate time interval

for df_logs in tqdm([train_logs, test_logs]):
    
    df_logs['timestamp'] = df_logs['time_diff']
    # Calculate time intervals in seconds from the start
    df_logs['time_interval'] = (df_logs['timestamp'] - df_logs.groupby('id')['timestamp'].transform('min'))


  0%|          | 0/2 [00:00<?, ?it/s]

In [58]:

'''
SD number of keystrokes per 30 s. Variance of the number of keystrokes in
every 15, 30, 45, 60, 120, and 180 s window
'''

for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):

    for second in tqdm([5, 8, 10, 15, 30, 45, 60, 80, 120, 180]):
        # Calculate number of keystrokes per 30-second intervals
        df_logs[f'{second}s_interval'] = (df_logs['time_interval'] // second).astype(int)

        df[f'sd_keystrokes_{second}s'] = df.id.map(df_logs.groupby(['id', f'{second}s_interval']).size().groupby('id').std())
        df[f'mean_keystrokes_{second}s'] = df.id.map(df_logs.groupby(['id', f'{second}s_interval']).size().groupby('id').mean())
        df[f'max_keystrokes_{second}s'] = df.id.map(df_logs.groupby(['id', f'{second}s_interval']).size().groupby('id').max())
        df[f'min_keystrokes_{second}s'] = df.id.map(df_logs.groupby(['id', f'{second}s_interval']).size().groupby('id').min())
        df[f'median_keystrokes_{second}s'] = df.id.map(df_logs.groupby(['id', f'{second}s_interval']).size().groupby('id').median())
    

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [59]:
'''
Slope of the number of keystrokes per 30 s. The slope of the linear regression
applied to the sequence of keystrokes in every 30 s window
'''

for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):
    
    for second in tqdm([4, 5, 8, 10, 15, 20, 25, 30, 60, 80]):
        
        df_logs[f'{second}s_interval'] = (df_logs['time_interval'] // second).astype(int)

        # Calculate slope of keystrokes per seconds intervals for each user
        df[f'slope_keystrokes_{second}s'] = df['id'].map(train_logs.groupby(['id', f'{second}s_interval'])['event_id'].max().reset_index().groupby('id').apply(lambda x: linregress(x[f'{second}s_interval'], x['event_id']).slope))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]

In [60]:
'''
Entropy of the number of keystrokes per 30 s. Maximum likelihood estimation
of Shannon entropy for the number of keystrokes in every 30 s window,
divided by the total number of keystrokes
'''

for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):
    
    for second in tqdm([5, 8, 10, 15, 25, 30, 45, 60]):
        df_logs[f'{second}s_interval'] = (df_logs['time_interval'] // second).astype(int)
        
        df[f'entropy_keystrokes_{second}s'] = df['id'].map(df_logs.groupby('id')[f'{second}s_interval'].apply(lambda x: entropy(x.value_counts(normalize=True))))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/8 [00:00<?, ?it/s]

In [61]:
pass
'''
# NOT APPL
Uniformity of the number of keystrokes per 30 s. Jensen-Shannon Divergence
of a uniform distribution of keystrokes (every window: total number of
keystrokes divided by total number of windows) and the actual distribution of keystrokes per 30 s
'''
pass

In [62]:
'''
Local extreme number of keystrokes per 30 s. Number of times the sign of
the difference in the number of keystrokes between 30 s window changes, from
increasing to decreasing number of keystrokes or vice versa.

'''

def calculate_local_extrema(values):
    differences = np.diff(values)
    num_extrema = np.sum(np.diff(np.sign(differences)) != 0)
    return num_extrema

for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):
    
    # Calculate local extrema for secend keystrokes per user
    for second in tqdm([3, 5, 8, 10, 15, 30, 40, 50, 60]):
        df_logs[f'{second}s_interval'] = (df_logs['time_interval'] // second).astype(int)
        
        df[f'{second}s_extrema_per_user'] = df['id'].map(df_logs.groupby('id')[f'{second}s_interval'].apply(calculate_local_extrema))

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

  0%|          | 0/9 [00:00<?, ?it/s]

In [63]:
'''
Mean and SD distance 30 s windows of more than one keystroke. Distance
between 30 s windows with more than one keystroke, giving a measure of the
amount and variance of long pauses.
'''

# Function to calculate mean and standard deviation of distances between 30 s windows with more than one keystroke

def calculate_distance_stats(values):
    keystroke_indices = np.where(values > 1)[0]  # Get indices where keystrokes > 1
    distances = np.diff(keystroke_indices) if len(keystroke_indices) > 1 else np.array([0])
    return np.mean(distances), np.std(distances) , np.sum(distances), np.max(distances)

for df, df_logs in tqdm([(train_feats, train_logs), (test_feats, test_logs)]):

    # Calculate local extrema for secend keystrokes per user
    for second in tqdm([3, 5, 8, 10, 15, 20, 30, 40, 50, 60, 80]):
        
        df_logs[f'{second}s_interval'] = (df_logs['time_interval'] // second).astype(int)
        
        # mean of distances for each user
        df[f'{second}_distance_stats_mean'] = df.id.map(df_logs.groupby('id')[f'{second}s_interval'].apply(calculate_distance_stats).apply(pd.Series)[0])

        # standard deviation of distances for each user
        df[f'{second}_distance_stats_std'] = df.id.map(df_logs.groupby('id')[f'{second}s_interval'].apply(calculate_distance_stats).apply(pd.Series)[1])

        # mean of distances for each user
        df[f'{second}_distance_stats_sum'] = df.id.map(df_logs.groupby('id')[f'{second}s_interval'].apply(calculate_distance_stats).apply(pd.Series)[2])

        # standard deviation of distances for each user
        df[f'{second}_distance_stats_max'] = df.id.map(df_logs.groupby('id')[f'{second}s_interval'].apply(calculate_distance_stats).apply(pd.Series)[3])


  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

  0%|          | 0/11 [00:00<?, ?it/s]

###  Features related to other events

In [64]:
pass

'''
Features related to other events. Lastly, we also included five keystroke features
related to non-typing or ’other’ events to get a broader view of students’ writing
behavior. Other events are all events which do not result in a character being typed;
these include mouse events, insertions, replacements, focus events, control, function,
and navigation keys (Baaijen & Galbraith, 2018). Since typos are extremely
common, these were not counted as an ’other’ event. We defined a typo as a revision
within a word, consisting of a maximum of three backspace or delete keystrokes,
and where the IKIs of the delete or backspace keystrokes are shorter than two SD
from the mean IKI in the copy task.
'''

'''
• Number of focus shifts to translation or task. Since focus shifts other than to
task or translation (e.g., to social media websites) were rare, we only included
those two types of focus shifts: the number of times the focus shifted towards
an online dictionary or translation web page, and the number of times the focus
shifted towards the task (Leijten et al., 2019).
• Mean and SD cut/paste/jump events. Time spent on cut (selection followed by
a keystroke or insertion), paste (insertion), and jump (mouse click resulting in
change in position in the document) events (Deane, 2014).
• Percentage of time spent on other events. Time spent on other events, divided
by the total time spent (Baaijen & Galbraith, 2018).
'''
pass

In [65]:
del preprocess_train
del preprocess_test
del silver_bullet_train
del silver_bullet_test
del paper_feature
del sentences_logs
del time_between_sentences_metrics
del IKI_between_sentences_metrics
del word_logs
del time_between_word_metrics
del IKI_between_word_metrics
del word_logs_longs_than_2std_pause
del multiple_backspace_duration_mean
del multiple_backspace_duration_std 
del multiple_backspace_duration_sum 
del multiple_backspace_duration_min 
del multiple_backspace_duration_max 
del single_backspace_duration_mean
del single_backspace_duration_std 
del single_backspace_duration_sum 
del single_backspace_duration_min 
del single_backspace_duration_max 
del single_backspace_duration 
del multiple_backspace_duration

gc.collect()

44

In [66]:
train_feats.shape, test_feats.shape

((2471, 911), (3, 910))

In [67]:
train_feats.to_csv('train_910_features.csv',index=False)

# Feature Interaction

In [68]:
for dt in [train_feats, test_feats]:
    
    dt['word_len_ge_6_count_mp'] = dt['word_len_ge_6_count']*dt['pauses_1_sec']
    dt['word_len_ge_6_count_mt'] = dt['word_len_ge_6_count']*dt['time_diff_comma_sum']
    dt['word_len_ge_6_count_mi'] = dt['word_len_ge_6_count']*dt['time_diff_Leftclick_median']
    dt['word_len_ge_6_count_md'] = dt['word_len_ge_6_count']*dt['down_event_8_cnt']
    dt['word_len_ge_6_count_mk'] = dt['word_len_ge_6_count']/dt['inter_key_median_lantency']
    
    dt['sent_len_ge_50_count_IKI_ median_Divid'] = dt['sent_len_ge_50_count'] / (dt['IKI_median'] + 1)
    dt['sent_word_count_mean_paragraph_word_count_mean_Divid'] = dt['sent_word_count_mean'] / (dt['paragraph_word_count_mean'] + 1)
    dt['sent_word_count_mean_cursor_position_diff_Enter_sum_Divid'] = dt['sent_word_count_mean']/dt['cursor_position_diff_Enter_sum'] + 1
    dt['down_event_8_cnt_cursor_position_diff_Enter_sum_Divid'] = dt['down_event_8_cnt'] / (dt['cursor_position_diff_Enter_sum'] + 1)
    dt['down_event_8_cnt_pauses_1_sec_Divid'] = dt['down_event_8_cnt'] / (dt['pauses_1_sec'] + 1)
    dt['word_count_diff_Input_mean_sent_len_std_Divid'] = dt['word_count_diff_Input_mean'] / (dt['sent_len_std'] + 1)
    dt['word_count_diff_Input_mean_paragraph_word_count_mean_Divid'] = dt['word_count_diff_Input_mean'] / (dt['paragraph_word_count_mean'] + 1)
    dt['word_count_diff_Input_mean_cursor_position_diff_Enter_sum_Divid'] = dt['word_count_diff_Input_mean'] / (dt['cursor_position_diff_Enter_sum'] + 1)
    dt['sent_len_std_cursor_position_diff_Leftclick_max_Divid'] = dt['sent_len_std'] / (dt['cursor_position_diff_Leftclick_max'] + 1)
    dt['sent_len_std_cursor_position_diff_Enter_sum_Divid'] = dt['sent_len_std'] / (dt['cursor_position_diff_Enter_sum'] + 1)
    dt['IKI_ median_sent_len_std_Divid'] = dt['IKI_median'] / (dt['sent_len_std'] + 1)
    dt['paragraph_word_count_mean_pauses_1_sec_Divid'] = dt['paragraph_word_count_mean'] / (dt['pauses_1_sec'] + 1)
    dt['word_len_max_IKI_ median_Divid'] = dt['word_len_max'] / (dt['IKI_median'] + 1)
    dt['word_len_max_pauses_1_sec_Divid'] = dt['word_len_max'] / (dt['pauses_1_sec'] + 1)
    dt['input_word_length_mean_IKI_ median_Divid'] = dt['input_word_length_mean'] / (dt['IKI_median'] + 1)
    dt['input_word_length_mean_cursor_position_diff_Enter_sum_Divid'] = dt['input_word_length_mean'] / (dt['cursor_position_diff_Enter_sum'] + 1)
    
    dt['sent_len_ge_50_count_time_diff_Input_mean_Div'] = dt['sent_len_ge_50_count'] / (dt['time_diff_Input_mean'] + 1)
    dt['P-bursts_count_time_diff_Input_mean_Div'] = dt['P-bursts_count'] / (dt['time_diff_Input_mean'] + 1)
    dt['pauses_1_sec_P-bursts_count_Divv'] = dt['pauses_1_sec'] / (dt['P-bursts_count'] + 1)
    dt['P-bursts_count_time_diff_abostrove_max_Div'] = dt['P-bursts_count'] / (dt['time_diff_abostrove_max'] + 1)
    dt['text_change_4_cnt_sent_len_ge_50_count_Div'] = dt['text_change_4_cnt'] / (dt['sent_len_ge_50_count'] + 1)
    dt['down_event_5_cnt_sent_len_ge_50_count_Div'] = dt['down_event_5_cnt'] / (dt['sent_len_ge_50_count'] + 1)
    dt['P-bursts_median_sent_len_ge_50_count_Div'] = dt['P-bursts_median'] / (dt['sent_len_ge_50_count'] + 1)



    dt['sent_len_std_cursor_position_diff_Enter_sum_Div'] = dt['sent_len_std'] / (dt['cursor_position_diff_Enter_sum'] + 1)
    dt['sent_len_std_time_diff_Backspace_median_Div'] = dt['sent_len_std'] / (dt['time_diff_Backspace_median'] + 1)
    dt['cursor_position_diff_Enter_sum_time_diff_Backspace_median_Div'] = dt['cursor_position_diff_Enter_sum'] / dt['time_diff_Backspace_median']
    dt['text_change_3_cnt_time_diff_Backspace_median_Div'] = dt['text_change_3_cnt'] / (dt['time_diff_Backspace_median'] + 1)
    dt['word_len_kurt_cursor_position_diff_Enter_sum_Div'] = dt['word_len_kurt'] / (dt['cursor_position_diff_Enter_sum'] + 1)
    dt['sent_len_median_cursor_position_diff_Enter_sum_Div'] = dt['sent_len_median'] / (dt['cursor_position_diff_Enter_sum'] + 1)
    dt['sent_len_median_time_diff_Backspace_median_Div'] = dt['sent_len_median'] / (dt['time_diff_Backspace_median'] + 1)
    dt['sent_len_median_paragraph_count_Div'] = dt['sent_len_median'] / (dt['paragraph_count'] + 1)
    dt['word_count_diff_Input_mean_sent_len_median_Div'] = dt['word_count_diff_Input_mean'] / (dt['sent_len_median'] + 1)
    dt['word_count_diff_Input_mean_text_change_3_cnt_Div'] = dt['word_count_diff_Input_mean'] / (dt['text_change_3_cnt'] + 1)
    dt['word_count_diff_Input_mean_word_len_max_Div'] = dt['word_count_diff_Input_mean'] / (dt['word_len_max'] + 1)
    dt['word_count_diff_Input_mean_paragraph_count_Div'] = dt['word_count_diff_Input_mean'] / (dt['paragraph_count'] + 1)
    dt['text_change_3_cnt_paragraph_len_last_Div'] = dt['text_change_3_cnt'] / (dt['paragraph_len_last'] + 1)

    dt['word_len_std_word_count_diff_Input_mean_MUl'] = dt['word_len_std'] * dt['word_count_diff_Input_mean']
    dt['word_len_ge_5_count_IKI_median_MUl'] = dt['word_len_ge_5_count'] * dt['IKI_median']
    dt['time_diff_Remove/Cut_median_word_len_std_DIVV'] = dt['time_diff_Remove/Cut_median'] / (dt['word_len_std'] +1)
    dt['text_change_3_cnt_punct_cnt_DIVV'] = dt['text_change_3_cnt'] / (dt['punct_cnt'] +1)
    

    for one, two in [(9,10),(8,9),(7,8), (6,7),(5,6),(4,5)]:
        
        dt[f'word_len_ge_{one}_count_word_len_ge_{two}_count_SUBB'] = dt[f'word_len_ge_{one}_count'] - dt[f'word_len_ge_{two}_count']
        dt[f'word_len_ge_{one}_count_word_len_ge_{two}_count_MULT'] = dt[f'word_len_ge_{one}_count'] + dt[f'word_len_ge_{two}_count']

    for o,t in [(1,3), (2,3),(2,1)]:
        dt[f'pauses_{o}_sec_pauses_{t}_sec_DIVV'] = dt[f'pauses_{o}_sec'] / (dt[f'pauses_{t}_sec'] + 1)
        dt[f'pauses_{o}_sec_pauses_{t}_sec_SUB'] = np.abs(dt[f'pauses_{o}_sec'] - dt[f'pauses_{t}_sec'])
        
  
    dt['IKI_median_sent_len_ge_80_count_MULT'] = dt['IKI_median'] * dt['sent_len_ge_80_count']
    dt['IKI_median_change_3_cnt_MULT'] = dt['IKI_median'] * dt['text_change_3_cnt']
    dt['IKI_median_word_len_std_MULT'] = dt['IKI_median'] * dt['word_len_std']
    dt['IKI_median_paragraph_len_max_MULT'] = dt['IKI_median'] * dt['paragraph_len_max']
    dt['IKI_median_pauses_half_sec_MULT'] = dt['IKI_median'] * dt['pauses_half_sec']
    
    dt['pauses_half_sec_word_len_mean_MULT'] = dt['pauses_half_sec'] / (dt['word_len_mean'] +1)
    dt['pauses_half_sec_sent_word_count_first_MULT'] = dt['pauses_half_sec'] / (dt['sent_word_count_first'] +1)
    
    dt['pauses_half_sec_word_len_mean_MULT'] = dt['pauses_half_sec'] * dt['word_len_mean'] 
    dt['pauses_half_sec_sent_word_count_first_MULT'] = dt['pauses_half_sec'] * dt['sent_word_count_first'] 
    
    dt['word_len_sem_pauses_1_half_sec_MULT'] = dt['word_len_sem'] * dt['pauses_1_half_sec']
    dt['word_len_sem_text_change_3_cnt_MULT'] = dt['word_len_sem'] * dt['text_change_3_cnt']
    dt['word_len_sem_sent_len_ge_75_count_MULT'] = dt['word_len_sem'] * dt['sent_len_ge_75_count']
    
    dt['inter_key_median_lantency_sent_len_ge_80_count_MULT'] = dt['inter_key_median_lantency'] * dt['sent_len_ge_80_count']
    dt['inter_key_median_lantency_text_change_3_cnt_MULT'] = dt['inter_key_median_lantency'] * dt['text_change_3_cnt']
    
    dt['sd_keystrokes_180s_sent_word_count_max_DIV'] = dt['sd_keystrokes_180s'] / (dt['sent_word_count_max'] +1)
    dt['sd_keystrokes_180s_mean_keystrokes_5s_DIV'] = dt['sd_keystrokes_180s'] / (dt['mean_keystrokes_5s'] +1)
    dt['sd_keystrokes_180s_paragraph_word_count_q1_DIV'] = dt['sd_keystrokes_180s'] / (dt['paragraph_word_count_q1'] +1)
    dt['sd_keystrokes_180s_60s_extrema_per_user_DIV'] = dt['sd_keystrokes_180s'] /( dt['60s_extrema_per_user'] +1)
    
    dt['Percentage_of_I_bursts_activity_2_cnt_DIVV'] = dt['Percentage_of_I_bursts'] / (dt['activity_2_cnt'] +1)
    dt['Percentage_of_I_bursts_sent_len_median_DIVV'] = dt['Percentage_of_I_bursts'] / (dt['sent_len_median'] +1)
    dt['Percentage_of_I_bursts_paragraph_word_count_median_DIVV'] = dt['Percentage_of_I_bursts'] / (dt['paragraph_word_count_median'] +1)
    
train_feats.shape

(2471, 993)

In [69]:
train_feats.to_csv('train_f_with_993.csv',index=False)
# out_path = '/kaggle/working/train_f_with_993.csv'
# train_feats = pd.read_csv(out_path)
# train_feats.head()

In [70]:
# test_feats = train_feats.head(3).drop('score',axis=1)

In [71]:
# # if model == 'svm'
# inf_columns = [col for col in train_feats.select_dtypes(exclude=object).columns if np.isinf(train_feats[col]).sum() > 0]
# numerical_feature = list(train_feats.select_dtypes(exclude=object).columns)
# numerical_feature = [ f for f in numerical_feature if f not in inf_columns]
# numerical_feature.remove('score')

# # uncomment this if it first try
# sc = StandardScaler() # MinMaxScaler or StandardScaler if gbdt
# train_feats[numerical_feature] = sc.fit_transform(train_feats[numerical_feature])
# test_feats[numerical_feature] = sc.transform(test_feats[numerical_feature])

In [72]:
# # if model == 'ridge'

# inf_columns = [col for col in train_feats.select_dtypes(exclude=object).columns if np.isinf(train_feats[col]).sum() > 0]
# numerical_feature = list(train_feats.select_dtypes(exclude=object).columns)
# numerical_feature = [ f for f in numerical_feature if f not in inf_columns]
# numerical_feature.remove('score')

# train_feats[numerical_feature] = np.log1p(train_feats[numerical_feature])
# test_feats[numerical_feature] = np.log1p(test_feats[numerical_feature])

# inf_columns = [col for col in numerical_feature if np.isinf(train_feats[col]).sum() > 0]


In [73]:
# train_feats = pd.read_csv('/kaggle/input/linking-preprocessing-papers-and-artical-features/train_f_with_993.csv')
# test_feats = train_feats.head(3)

train_df = train_feats.copy()
test_df = test_feats.copy()

In [74]:
train_feats.shape, test_feats.shape

((2471, 993), (3, 992))

# Feature Transformation

In [75]:
# # if model == 'svm' or 'ridge'
def transformation(k):
 
    global train_feats, test_feats
    
    train_feats = train_df.copy()
    test_feats = test_df.copy()

    inf_columns = [col for col in train_feats.select_dtypes(exclude=object).columns if np.isinf(train_feats[col]).sum() > 0]
    numerical_feature = list(train_feats.select_dtypes(exclude=object).columns)
    numerical_feature = [ f for f in numerical_feature if f not in inf_columns]
    numerical_feature.remove('score')
    
    if k =='MinMax':
        
        sc = MinMaxScaler() # MinMaxScaler or StandardScaler
        train_feats[numerical_feature] = sc.fit_transform(train_feats[numerical_feature])
        test_feats[numerical_feature] = sc.transform(test_feats[numerical_feature])
        
    elif k =='StanderScaler':
        
        sc = StandardScaler() # MinMaxScaler or StandardScaler
        train_feats[numerical_feature] = sc.fit_transform(train_feats[numerical_feature])
        test_feats[numerical_feature] = sc.transform(test_feats[numerical_feature])
        
    elif k =='Logrithm':
        
        train_feats[numerical_feature] = np.log1p(train_feats[numerical_feature])
        test_feats[numerical_feature] = np.log1p(test_feats[numerical_feature])
    
    elif k =='No_transfomer':
        pass
      

# Features and Models Selection

In [76]:
ridge_model = Ridge(alpha=0.05,solver='saga')

svr_model = SVR(C=1)

rfr_model = RandomForestRegressor(n_estimators=100,
                              max_depth=7,
                              verbose=0,
                              min_samples_split=4,
                              max_features = 0.25,
                              random_state=42)

lgbm_model = lgb.LGBMRegressor(num_leaves=8,
                          max_depth=4,
                          learning_rate=0.1,
                          n_estimators=1024,
                          subsample=0.005,
                          colsample_bytree=0.25,
                          reg_alpha=0.0005,
                          reg_lambda=0.1,
                          random_state=42,
                         metric=None)


xgb_param={
'reg_alpha': 0.001,
'reg_lambda': 2.5,
'colsample_bynode': 0.75,
'subsample': 0.85, 
'eta': 0.05, 
'max_depth': 3, 
'n_estimators': 1024,
'random_state': 42,
'eval_metric': 'rmse'
}
cbt_param = {
         'learning_rate': 0.1, 
          'depth': 5, 
          'l2_leaf_reg': 4, 
          'loss_function': 'MultiRMSE', 
          'eval_metric': 'MultiRMSE', 
          'task_type': 'CPU', 
          'iterations': 1024,
          'od_type': 'Iter',
          'boosting_type': 'Plain', 
          'bootstrap_type': 'Bayesian',
          'allow_const_label': True, 
          'random_state': 1
}

xgb_model = XGBRegressor(**xgb_param)
cbt_model = CatBoostRegressor(**cbt_param)

In [77]:
# path = 'Linking/20_Dec_lofo_feature_importance_SVR_6.csv'
# features = pd.read_csv(path)
# features_selected = list(features[features['importance_mean'] >=0].feature)

# print(f'{len(features_selected)} FEATURES selected from {train_feats.shape[1]} FEATURES ')

# this features extracted from lofo imporrtance
# Linking Preprocessing Papers and Artical Features LightGBM cv0.601

lgbm_features = ['word_len_ge_8_count_word_len_ge_9_count_MULT', 'word_len_sem_sent_len_ge_75_count_MULT', 'time_diff_comma_std', 'sent_len_ge_100_count', 'paragraph_word_count_min', 'cursor_position_diff_Input_sum', 'cursor_position_diff_dot_std', 'paragraph_len_sum', 'word_count_diff_Replace_sum', 'action_time_std', 'Number of revisions Remove/Cut 10 word', '50_distance_stats_mean', 'time_diff_abostrove_median', 'word_len_last', 'paragraph_len_q1', 'word_count_diff_abostrove_mean', 'cursor_position_diff_ArrowDown_std', 'cursor_position_change_5_std', 'time_diff_ArrowDown_max', 'word_count_change_50_std', 'word_count_change_5_std', 'word_count_change_1_max', 'time_diff_Unidentified_max', 'down_event_6_cnt', 'word_len_ge_7_count_word_len_ge_8_count_SUBB', 'time_diff_ArrowLeft_max', 'cursor_position_change_10_sum', 'cursor_position_diff_Space_mean', 'action_time_gap_5_std', 'action_time_sum', 'mean_keystrokes_120s', 'word_count_diff_Paste_std', 'mean_keystrokes_60s', 'time_diff_Shift_std', 'time_diff_Shift_max', 'sent_len_skew', 'sd_keystrokes_120s', 'time_diff_Nonproduction_median', 'std_pause_time', 'word_len_ge_9_count_word_len_ge_10_count_SUBB', 'leading-edge revisions_mean', 'input_word_length_skew', 'Number of revisions Remove/Cut 14 word', 'num_revision_bursts', 'action_time_gap_5_mean', 'time_diff_dot_max', 'cursor_position_diff_Remove/Cut_mean', 'entropy_keystrokes_5s', 'word_count_diff_Input_mean_paragraph_word_count_mean_Divid', '30_distance_stats_sum', 'word_count_sem', 'time_diff_Shift_median', '10_distance_stats_max', 'sent_len_ge_50_count_time_diff_Input_mean_Div', 'action_time_gap_10_mean', 'word_count_diff_Space_sum', 'pauses_3_sec', 'min_keystrokes_5s', 'cursor_position_diff_ArrowDown_max', 'down_event_8_cnt_pauses_1_sec_Divid', 'sent_word_count_mean', 'cursor_position_diff_Remove/Cut_median', 'Number of revisions Backspace 14 word', 'word_count_change_1_sum', 'cursor_position_diff_ArrowUp_max', 'word_count_diff_Paste_median', 'time_diff_Unidentified_mean', 'time_diff_Input_sum', 'P-bursts_max', 'cursor_position_change_5_sum', 'cursor_position_diff_Leftclick_sum', 'mean_keystrokes_80s', 'word_len_ge_4_count_word_len_ge_5_count_MULT', 'time_diff_Shift_sum', 'backspacing_time_skew', 'num_insertion_bursts', 'sent_len_median_time_diff_Backspace_median_Div', 'cursor_position_diff_Nonproduction_mean', 'inter_key_median_lantency', 'leading-edge revisions_max', 'cursor_position_diff_Input_mean', 'cursor_position_diff_Backspace_sum', 'time_diff_ArrowUp_sum', 'max_chars_per_burst', '10_distance_stats_mean', 'word_len_q1', 'cursor_position_diff_Backspace_mean', 'word_count_diff_q_mean', 'text_change', 'cursor_position_diff_ArrowDown_median', 'down_event_11_cnt', 'word_count_diff_Enter_std', 'cursor_position_change_1_sum', 'sequence_of_backspaces_std', 'cursor_position_sem', 'paragraph_len_mean', 'sent_len_max', 'cursor_position_change_100_max', 'word_len_ge_6_count_md', 'sequence_of_backspaces_sum', 'word_len_sem', 'action_time_gap_20_sum', '10s_extrema_per_user', 'word_len_ge_10_count', 'backspacing_time_max', 'word_len_ge_4_count', 'cursor_position_diff_Paste_median', 'cursor_position_change_2_max', 'action_time_gap_20_mean', 'text_change_8_cnt', 'cursor_position_diff_Shift_mean', 'median_keystrokes_10s', 'word_len_mean', 'P-bursts_count', 'down_event_2_cnt', 'IKI_sum', 'sent_word_count_sem', 'pauses_half_sec_sent_word_count_first_MULT', 'IKI_median_pauses_half_sec_MULT', 'word_count_diff_Remove/Cut_sum', 'cursor_position_change_2_sum', 'cursor_position_diff_Leftclick_max', 'time_diff_Remove/Cut_std', 'time_diff_Space_std', 'action_time_gap_50_min', '30_distance_stats_mean', 'word_count_diff_Paste_max', 'sent_len_first', 'cursor_position_diff_ArrowLeft_median', 'word_count_diff_comma_std', 'text_change_4_cnt', 'time_diff_dot_median', 'time_diff_Unidentified_sum', 'time_diff_Nonproduction_sum', 'text_change_12_cnt', 'cursor_position_diff_Leftclick_mean', 'up_time_quantile', 'Number of revisions Backspace 3 word', 'inter_key_median_lantency_text_change_3_cnt_MULT', 'Percentage_of_I_bursts_paragraph_word_count_median_DIVV', '80_distance_stats_std', 'paragraph_len_median', 'slope_keystrokes_80s', 'cursor_position_diff_ArrowRight_mean', 'cursor_position_diff_ArrowLeft_sum', 'word_count_diff_Delete_median', 'word_count_mean', 'long_pauses_sum', 'word_count_diff_Replace_mean', 'word_count_change_3_mean', 'sent_word_count_skew', 'sent_len_std_time_diff_Backspace_median_Div', 'down_event_12_cnt', 'word_count_diff_dot_sum', 'word_count_median', 'action_time_gap_50_sum', 'cursor_position_median', 'time_diff_Input_max', 'time_diff_ArrowDown_std', 'cursor_position_diff_Input_max', 'down_event_14_cnt', 'sent_len_ge_50_count']

# Linking Preprocessing Papers and Artical Features-RandomForestRegression cv0.615
rfr_features = ['word_len_ge_8_count_word_len_ge_9_count_MULT', 'word_len_sem_sent_len_ge_75_count_MULT', 'time_diff_comma_std', 'sent_len_ge_100_count', 'paragraph_word_count_min', 'cursor_position_diff_Input_sum', 'cursor_position_diff_dot_std', 'paragraph_len_sum', 'word_count_diff_Replace_sum', 'action_time_std', 'Number of revisions Remove/Cut 10 word', '50_distance_stats_mean', 'time_diff_abostrove_median', 'word_len_last', 'paragraph_len_q1', 'word_count_diff_abostrove_mean', 'cursor_position_diff_ArrowDown_std', 'cursor_position_change_5_std', 'time_diff_ArrowDown_max', 'word_count_change_50_std', 'word_count_change_5_std', 'word_count_change_1_max', 'time_diff_Unidentified_max', 'down_event_6_cnt', 'word_len_ge_7_count_word_len_ge_8_count_SUBB', 'time_diff_ArrowLeft_max', 'cursor_position_change_10_sum', 'cursor_position_diff_Space_mean', 'action_time_gap_5_std', 'action_time_sum', 'mean_keystrokes_120s', 'word_count_diff_Paste_std', 'mean_keystrokes_60s', 'time_diff_Shift_std', 'time_diff_Shift_max', 'sent_len_skew', 'sd_keystrokes_120s', 'time_diff_Nonproduction_median', 'std_pause_time', 'word_len_ge_9_count_word_len_ge_10_count_SUBB', 'leading-edge revisions_mean', 'input_word_length_skew', 'Number of revisions Remove/Cut 14 word', 'num_revision_bursts', 'action_time_gap_5_mean', 'time_diff_dot_max', 'cursor_position_diff_Remove/Cut_mean', 'entropy_keystrokes_5s', 'word_count_diff_Input_mean_paragraph_word_count_mean_Divid', '30_distance_stats_sum', 'word_count_sem', 'time_diff_Shift_median', '10_distance_stats_max', 'sent_len_ge_50_count_time_diff_Input_mean_Div', 'action_time_gap_10_mean', 'word_count_diff_Space_sum', 'pauses_3_sec', 'min_keystrokes_5s', 'cursor_position_diff_ArrowDown_max', 'down_event_8_cnt_pauses_1_sec_Divid', 'sent_word_count_mean', 'cursor_position_diff_Remove/Cut_median', 'Number of revisions Backspace 14 word', 'word_count_change_1_sum', 'cursor_position_diff_ArrowUp_max', 'word_count_diff_Paste_median', 'time_diff_Unidentified_mean', 'time_diff_Input_sum', 'P-bursts_max', 'cursor_position_change_5_sum', 'cursor_position_diff_Leftclick_sum', 'mean_keystrokes_80s', 'word_len_ge_4_count_word_len_ge_5_count_MULT', 'time_diff_Shift_sum', 'backspacing_time_skew', 'num_insertion_bursts', 'sent_len_median_time_diff_Backspace_median_Div', 'cursor_position_diff_Nonproduction_mean', 'inter_key_median_lantency', 'leading-edge revisions_max', 'cursor_position_diff_Input_mean', 'cursor_position_diff_Backspace_sum', 'time_diff_ArrowUp_sum', 'max_chars_per_burst', '10_distance_stats_mean', 'word_len_q1', 'cursor_position_diff_Backspace_mean', 'word_count_diff_q_mean', 'text_change', 'cursor_position_diff_ArrowDown_median', 'down_event_11_cnt', 'word_count_diff_Enter_std', 'cursor_position_change_1_sum', 'sequence_of_backspaces_std', 'cursor_position_sem', 'paragraph_len_mean', 'sent_len_max', 'cursor_position_change_100_max', 'word_len_ge_6_count_md', 'sequence_of_backspaces_sum', 'word_len_sem', 'action_time_gap_20_sum', '10s_extrema_per_user', 'word_len_ge_10_count', 'backspacing_time_max', 'word_len_ge_4_count', 'cursor_position_diff_Paste_median', 'cursor_position_change_2_max', 'action_time_gap_20_mean', 'text_change_8_cnt', 'cursor_position_diff_Shift_mean', 'median_keystrokes_10s', 'word_len_mean', 'P-bursts_count', 'down_event_2_cnt', 'IKI_sum', 'sent_word_count_sem', 'pauses_half_sec_sent_word_count_first_MULT', 'IKI_median_pauses_half_sec_MULT', 'word_count_diff_Remove/Cut_sum', 'cursor_position_change_2_sum', 'cursor_position_diff_Leftclick_max', 'time_diff_Remove/Cut_std', 'time_diff_Space_std', 'action_time_gap_50_min', '30_distance_stats_mean', 'word_count_diff_Paste_max', 'sent_len_first', 'cursor_position_diff_ArrowLeft_median', 'word_count_diff_comma_std', 'text_change_4_cnt', 'time_diff_dot_median', 'time_diff_Unidentified_sum', 'time_diff_Nonproduction_sum', 'text_change_12_cnt', 'cursor_position_diff_Leftclick_mean', 'up_time_quantile', 'Number of revisions Backspace 3 word', 'inter_key_median_lantency_text_change_3_cnt_MULT', 'Percentage_of_I_bursts_paragraph_word_count_median_DIVV', '80_distance_stats_std', 'paragraph_len_median', 'slope_keystrokes_80s', 'cursor_position_diff_ArrowRight_mean', 'cursor_position_diff_ArrowLeft_sum', 'word_count_diff_Delete_median', 'word_count_mean', 'long_pauses_sum', 'word_count_diff_Replace_mean', 'word_count_change_3_mean', 'sent_word_count_skew', 'sent_len_std_time_diff_Backspace_median_Div', 'down_event_12_cnt', 'word_count_diff_dot_sum', 'word_count_median', 'action_time_gap_50_sum', 'cursor_position_median', 'time_diff_Input_max', 'time_diff_ArrowDown_std', 'cursor_position_diff_Input_max', 'down_event_14_cnt', 'sent_len_ge_50_count']

# Linking Preprocessing Papers and Artical Features-Ridge cv0.591 min max
ridge_features = ['word_len_std', 'word_len_sem', 'paragraph_count', 'word_count_diff_Input_mean_text_change_3_cnt_Div', 'IKI_median', 'paragraph_len_max', 'sent_word_count_q3', 'inter_key_median_lantency', 'punct_cnt', 'time_diff_Space_median', 'cursor_position_diff_abostrove_median', 'time_diff_comma_sum', 'long_pauses_sum', 'word_count_diff_Space_mean', 'entropy_keystrokes_30s', 'sent_word_count_first', 'pauses_half_sec', 'word_len_ge_6_count_md', 'paragraph_len_last', 'paragraph_len_first', 'R-bursts_mean', 'time_diff_Backspace_sum', 'paragraph_len_min', 'word_count_diff_q_mean', 'sent_len_skew', 'word_len_mean', 'word_len_max', 'word_len_ge_6_count', 'paragraph_word_count_sum', 'pauses_1_sec_P-bursts_count_Divv', 'paragraph_len_kurt', 'time_diff_Remove/Cut_mean', 'time_diff_ArrowRight_max', 'cursor_position_change_10_std', 'pauses_1_sec', 'time_diff_Input_max', 'down_event_7_cnt', 'max_keystrokes_60s', 'sent_len_ge_60_count', 'cursor_position_diff_ArrowRight_median', 'sent_count', 'time_diff_comma_max', 'down_event_6_cnt', 'word_count_sem', 'R-bursts_median', 'text_change_3_cnt', 'backspacing_time_skew', 'word_count_diff_Input_mean', 'time_diff_q_mean', 'word_count_change_2_mean', 'mean_keystrokes_5s', 'sd_keystrokes_80s', 'time_diff_Shift_sum', 'cursor_position_diff_Enter_sum', 'cursor_position_diff_Delete_mean', 'down_event_8_cnt', 'word_count_diff_Delete_max', 'Number of revisions Backspace 50 word', 'cursor_position_diff_ArrowRight_max', 'median_keystrokes_8s', 'word_count_diff_Input_mean_sent_len_median_Div', 'word_count_diff_Space_sum', 'time_diff_Leftclick_median', 'cursor_position_diff_ArrowUp_median', 'cursor_position_diff_ArrowUp_max', 'time_diff_Replace_median', 'word_len_max_pauses_1_sec_Divid', 'leading-edge revisions_sum', 'down_time_median', 'cursor_position_diff_abostrove_sum', 'cursor_position_diff_dot_sum', 'word_len_ge_5_count', 'word_count_diff_Input_mean_word_len_max_Div', 'time_diff_abostrove_median', 'word_count_change_20_std', 'time_diff_abostrove_std', 'word_len_q3', 'sent_len_max', 'time_diff_ArrowRight_std', 'cursor_position_diff_ArrowUp_std', 'word_len_ge_3_count', 'action_time_gap_3_sum', 'action_time_gap_5_sum', 'cursor_position_diff_q_std', 'time_diff_ArrowLeft_median', 'time_diff_comma_median', 'action_time_gap_100_sum', 'time_diff_Nonproduction_max', 'action_time_gap_3_max', 'word_count_diff_Input_sum', 'action_time_gap_2_sum', 'word_count_change_10_std']

# Linking Preprocessing Papers and Artical Features-Ridge cv0.596 log transformation
log_ridge_features = ['punct_cnt', 'paragraph_word_count_max', 'sent_len_ge_75_count', 'time_diff_dot_sum', 'word_len_ge_3_count', 'word_len_max_pauses_1_sec_Divid', 'time_diff_Input_max', 'paragraph_word_count_std', 'time_diff_Shift_mean', 'paragraph_len_mean', 'time_diff_dot_mean', 'product_to_keys', 'action_time_gap_50_sum', 'sent_len_q1', 'sent_word_count_mean_paragraph_word_count_mean_Divid', 'long_pauses_std', 'time_diff_Enter_sum', 'sent_len_ge_120_count', 'paragraph_len_q3', 'time_diff_Enter_mean', 'slope_keystrokes_30s', 'sent_word_count_sum', 'time_diff_ArrowLeft_median', 'cursor_position_diff_abostrove_median', '3s_extrema_per_user', 'slope_keystrokes_8s', 'time_diff_Paste_max', 'time_diff_Space_std', 'down_event_11_cnt', 'time_diff_Paste_sum', 'action_time_gap_20_mean', 'P-bursts_mean', 'P-bursts_last', 'word_count_diff_Delete_std', 'pauses_1_sec_pauses_3_sec_SUB', 'down_event_8_cnt_pauses_1_sec_Divid', 'action_time_gap_10_std', 'word_len_sum', 'IKI_median_sent_len_ge_80_count_MULT', 'max_keystrokes_10s', 'action_time_gap_50_max', 'cursor_position_diff_Delete_std', 'action_time_gap_5_std', 'mean_keystrokes_80s', 'cursor_position_change_50_std', 'cursor_position_diff_abostrove_std', 'word_count_change_5_std', 'R-bursts_mean', '60_distance_stats_max', 'sent_word_count_first', 'pauses_half_sec_sent_word_count_first_MULT', 'word_len_ge_15_count', 'cursor_position_diff_abostrove_sum', 'max_keystrokes_180s', 'word_count_diff_Remove/Cut_std', '5s_extrema_per_user', 'max_keystrokes_5s', 'time_diff_Backspace_median', 'word_count_change_20_std', 'sent_len_sum', 'pauses_half_sec', 'action_time_gap_10_max', 'sent_len_median_time_diff_Backspace_median_Div', 'inter_key_median_lantency', 'cursor_position_diff_Space_mean', 'median_keystrokes_180s', 'time_diff_CapsLock_mean', 'time_diff_CapsLock_sum', 'word_count_diff_Input_mean_word_len_max_Div', 'cursor_position_change_2_std', 'word_time_ratio']

# Linking Preprocessing Papers and Artical Features-SVR cv0.588
svr_features = ['word_count_diff_Input_mean', 'IKI_median', 'time_diff_Space_median', 'backspacing_time_skew', 'word_count_diff_Input_mean_sent_len_median_Div', 'word_count_diff_Input_mean_text_change_3_cnt_Div', 'pauses_1_sec_P-bursts_count_Divv', 'time_diff_Backspace_sum', 'word_len_std', 'time_diff_Backspace_mean', 'text_change_3_cnt', 'pauses_half_sec', 'word_len_ge_5_count', 'time_diff_Space_mean', 'sent_len_ge_75_count', 'cursor_position_diff_Leftclick_sum', 'up_event', 'paragraph_len_last', 'word_count_diff_Input_mean_paragraph_count_Div', 'word_len_ge_7_count', 'max_keystrokes_8s', 'time_diff_q_sum', 'time_diff_Input_sum', 'punct_cnt', 'time_diff_Paste_std', 'word_len_ge_6_count_mp', 'inter_key_median_lantency', 'pauses_1_half_sec', 'paragraph_word_count_q1', 'paragraph_len_q1', 'time_diff_q_median', 'paragraph_len_median', 'sent_word_count_first', 'time_diff_Input_median', 'paragraph_len_min', 'R-bursts_mean', 'paragraph_word_count_median', 'pauses_3_sec', 'down_event_8_cnt', 'time_diff_Enter_median', 'Percentage_of_I_bursts', 'paragraph_word_count_sum', 'down_time_min', 'paragraph_word_count_min', 'time_diff_Remove/Cut_median', 'down_event_1_cnt', 'word_count_diff_Input_mean_paragraph_word_count_mean_Divid', 'leading-edge revisions_count', '60s_extrema_per_user', 'time_diff_Shift_sum', 'text_change', 'paragraph_len_first', 'sent_word_count_mean_paragraph_word_count_mean_Divid', 'product_to_keys', 'sent_word_count_min', 'word_count_diff_Paste_std', 'action_time_gap_20_max', 'sent_word_count_last', 'sent_len_ge_150_count', 'sent_len_ge_60_count', 'paragraph_word_count_first', 'sent_len_last', 'time_diff_Replace_mean']

# Pruning Features
feature_selected = ['activity_0_cnt', 'activity_1_cnt', 'activity_2_cnt', 'activity_3_cnt', 'activity_4_cnt', 'text_change_0_cnt', 'text_change_1_cnt', 'text_change_2_cnt', 'text_change_3_cnt', 'text_change_4_cnt', 'text_change_5_cnt', 'text_change_6_cnt', 'text_change_7_cnt', 'text_change_8_cnt', 'text_change_9_cnt', 'text_change_10_cnt', 'text_change_11_cnt', 'text_change_12_cnt', 'text_change_13_cnt', 'down_event_0_cnt', 'down_event_1_cnt', 'down_event_2_cnt', 'down_event_3_cnt', 'down_event_4_cnt', 'down_event_5_cnt', 'down_event_6_cnt', 'down_event_7_cnt', 'down_event_8_cnt', 'down_event_9_cnt', 'down_event_10_cnt', 'down_event_11_cnt', 'down_event_12_cnt', 'down_event_13_cnt', 'down_event_14_cnt', 'down_event_15_cnt', 'up_event_0_cnt', 'input_word_length_median', 'input_word_length_skew', 'down_time_mean', 'up_time_mean', 'down_time_std', 'up_time_std', 'cursor_position_std', 'word_count_std', 'down_time_median', 'up_time_median', 'action_time_median', 'cursor_position_median', 'word_count_median', 'down_time_min', 'up_time_min', 'word_count_min', 'down_time_max', 'down_time_quantile', 'up_time_quantile', 'activity', 'down_event', 'up_event', 'text_change', 'inter_key_largest_lantency', 'inter_key_median_lantency', 'mean_pause_time', 'std_pause_time', 'total_pause_time', 'pauses_half_sec', 'pauses_1_sec', 'pauses_1_half_sec', 'pauses_2_sec', 'pauses_3_sec', 'P-bursts_mean', 'P-bursts_std', 'P-bursts_count', 'P-bursts_median', 'P-bursts_max', 'P-bursts_first', 'P-bursts_last', 'R-bursts_mean', 'R-bursts_std', 'R-bursts_median', 'R-bursts_max', 'R-bursts_first', 'R-bursts_last', 'keys_per_second', 'product_to_keys', 'event_id_max', 'up_time_max', 'action_time_max', 'action_time_min', 'action_time_mean', 'action_time_std', 'action_time_sem', 'action_time_sum', 'action_time_skew' , 'cursor_position_nunique', 'cursor_position_max', 'cursor_position_sem', 'cursor_position_mean', 'word_count_nunique', 'word_count_max', 'word_count_sem', 'word_count_mean', 'action_time_gap_1_max', 'action_time_gap_1_min', 'action_time_gap_1_mean', 'action_time_gap_1_std', 'action_time_gap_1_sum', 'action_time_gap_2_max', 'action_time_gap_2_min', 'action_time_gap_2_mean', 'action_time_gap_2_std', 'action_time_gap_2_sum', 'action_time_gap_3_max', 'action_time_gap_3_min', 'action_time_gap_3_mean', 'action_time_gap_3_std', 'action_time_gap_3_sum', 'action_time_gap_5_max', 'action_time_gap_5_min', 'action_time_gap_5_mean', 'action_time_gap_5_std', 'action_time_gap_5_sum', 'action_time_gap_10_max', 'action_time_gap_10_min', 'action_time_gap_10_mean', 'action_time_gap_10_std', 'action_time_gap_10_sum', 'action_time_gap_20_max', 'action_time_gap_20_min', 'action_time_gap_20_mean', 'action_time_gap_20_std', 'action_time_gap_20_sum', 'action_time_gap_50_max', 'action_time_gap_50_min', 'action_time_gap_50_mean', 'action_time_gap_50_std', 'action_time_gap_50_sum', 'action_time_gap_100_max', 'action_time_gap_100_min', 'action_time_gap_100_mean', 'action_time_gap_100_std', 'action_time_gap_100_sum', 'cursor_position_change_1_max', 'cursor_position_change_1_mean', 'cursor_position_change_1_std', 'cursor_position_change_1_sum', 'cursor_position_change_2_max', 'cursor_position_change_2_mean', 'cursor_position_change_2_std', 'cursor_position_change_2_sum', 'cursor_position_change_3_max', 'cursor_position_change_3_mean', 'cursor_position_change_3_std', 'cursor_position_change_3_sum', 'cursor_position_change_5_max', 'cursor_position_change_5_mean', 'cursor_position_change_5_std', 'cursor_position_change_5_sum', 'cursor_position_change_10_max', 'cursor_position_change_10_mean', 'cursor_position_change_10_std', 'cursor_position_change_10_sum', 'cursor_position_change_20_max', 'cursor_position_change_20_mean', 'cursor_position_change_20_std', 'cursor_position_change_20_sum', 'cursor_position_change_50_max', 'cursor_position_change_50_mean', 'cursor_position_change_50_std', 'cursor_position_change_50_sum', 'cursor_position_change_100_max', 'cursor_position_change_100_mean', 'cursor_position_change_100_std', 'cursor_position_change_100_sum', 'word_count_change_1_max', 'word_count_change_1_mean', 'word_count_change_1_std', 'word_count_change_1_sum', 'word_count_change_2_max', 'word_count_change_2_mean', 'word_count_change_2_std', 'word_count_change_2_sum', 'word_count_change_3_max', 'word_count_change_3_mean', 'word_count_change_3_std', 'word_count_change_3_sum', 'word_count_change_5_max', 'word_count_change_5_mean', 'word_count_change_5_std', 'word_count_change_5_sum', 'word_count_change_10_max', 'word_count_change_10_mean', 'word_count_change_10_std', 'word_count_change_10_sum', 'word_count_change_20_max', 'word_count_change_20_mean', 'word_count_change_20_std', 'word_count_change_20_sum', 'word_count_change_50_max', 'word_count_change_50_mean', 'word_count_change_50_std', 'word_count_change_50_sum', 'word_count_change_100_max', 'word_count_change_100_mean', 'word_count_change_100_std', 'word_count_change_100_sum', 'punct_cnt', 'input_word_count', 'input_word_length_mean', 'input_word_length_max', 'input_word_length_std', 'word_time_ratio', 'word_event_ratio', 'event_time_ratio', 'time_diff_q_std', 'time_diff_Space_std', 'time_diff_Backspace_std', 'time_diff_Shift_std', 'time_diff_ArrowRight_std', 'time_diff_Leftclick_std', 'time_diff_ArrowLeft_std', 'time_diff_dot_std', 'time_diff_comma_std', 'time_diff_ArrowDown_std', 'time_diff_ArrowUp_std', 'time_diff_Enter_std', 'time_diff_CapsLock_std', 'time_diff_abostrove_std', 'time_diff_Delete_std', 'time_diff_Unidentified_std', 'time_diff_q_mean', 'time_diff_Space_mean', 'time_diff_Backspace_mean', 'time_diff_Shift_mean', 'time_diff_ArrowRight_mean', 'time_diff_Leftclick_mean', 'time_diff_ArrowLeft_mean', 'time_diff_dot_mean', 'time_diff_comma_mean', 'time_diff_ArrowDown_mean', 'time_diff_ArrowUp_mean', 'time_diff_Enter_mean', 'time_diff_CapsLock_mean', 'time_diff_abostrove_mean', 'time_diff_Delete_mean', 'time_diff_Unidentified_mean', 'time_diff_q_sum', 'time_diff_Space_sum', 'time_diff_Backspace_sum', 'time_diff_Shift_sum', 'time_diff_ArrowRight_sum', 'time_diff_Leftclick_sum', 'time_diff_ArrowLeft_sum', 'time_diff_dot_sum', 'time_diff_comma_sum', 'time_diff_ArrowDown_sum', 'time_diff_ArrowUp_sum', 'time_diff_Enter_sum', 'time_diff_CapsLock_sum', 'time_diff_abostrove_sum', 'time_diff_Delete_sum', 'time_diff_Unidentified_sum', 'time_diff_q_median', 'time_diff_Space_median', 'time_diff_Backspace_median', 'time_diff_Shift_median', 'time_diff_ArrowRight_median', 'time_diff_Leftclick_median', 'time_diff_ArrowLeft_median', 'time_diff_dot_median', 'time_diff_comma_median', 'time_diff_ArrowDown_median', 'time_diff_ArrowUp_median', 'time_diff_Enter_median', 'time_diff_CapsLock_median', 'time_diff_abostrove_median', 'time_diff_Delete_median', 'time_diff_Unidentified_median', 'time_diff_q_max', 'time_diff_Space_max', 'time_diff_Backspace_max', 'time_diff_Shift_max', 'time_diff_ArrowRight_max', 'time_diff_Leftclick_max', 'time_diff_ArrowLeft_max', 'time_diff_dot_max', 'time_diff_comma_max', 'time_diff_ArrowDown_max', 'time_diff_ArrowUp_max', 'time_diff_Enter_max', 'time_diff_CapsLock_max', 'time_diff_abostrove_max', 'time_diff_Delete_max', 'time_diff_Unidentified_max', 'cursor_position_diff_q_std', 'cursor_position_diff_Space_std', 'cursor_position_diff_Backspace_std', 'cursor_position_diff_Shift_std', 'cursor_position_diff_ArrowRight_std', 'cursor_position_diff_Leftclick_std', 'cursor_position_diff_ArrowLeft_std', 'cursor_position_diff_dot_std', 'cursor_position_diff_comma_std', 'cursor_position_diff_ArrowDown_std', 'cursor_position_diff_ArrowUp_std', 'cursor_position_diff_Enter_std', 'cursor_position_diff_abostrove_std', 'cursor_position_diff_Delete_std', 'cursor_position_diff_q_mean', 'cursor_position_diff_Space_mean', 'cursor_position_diff_Backspace_mean', 'cursor_position_diff_Shift_mean', 'cursor_position_diff_ArrowRight_mean', 'cursor_position_diff_Leftclick_mean', 'cursor_position_diff_ArrowLeft_mean', 'cursor_position_diff_dot_mean', 'cursor_position_diff_comma_mean', 'cursor_position_diff_ArrowDown_mean', 'cursor_position_diff_ArrowUp_mean', 'cursor_position_diff_Enter_mean', 'cursor_position_diff_abostrove_mean', 'cursor_position_diff_Delete_mean', 'cursor_position_diff_q_sum', 'cursor_position_diff_Space_sum', 'cursor_position_diff_Backspace_sum', 'cursor_position_diff_Shift_sum', 'cursor_position_diff_ArrowRight_sum', 'cursor_position_diff_Leftclick_sum', 'cursor_position_diff_ArrowLeft_sum', 'cursor_position_diff_dot_sum', 'cursor_position_diff_comma_sum', 'cursor_position_diff_ArrowDown_sum', 'cursor_position_diff_ArrowUp_sum', 'cursor_position_diff_Enter_sum', 'cursor_position_diff_abostrove_sum', 'cursor_position_diff_Delete_sum', 'cursor_position_diff_ArrowRight_median', 'cursor_position_diff_Leftclick_median', 'cursor_position_diff_ArrowLeft_median', 'cursor_position_diff_ArrowDown_median', 'cursor_position_diff_ArrowUp_median', 'cursor_position_diff_abostrove_median', 'cursor_position_diff_Delete_median', 'cursor_position_diff_Space_max', 'cursor_position_diff_Backspace_max', 'cursor_position_diff_Shift_max', 'cursor_position_diff_ArrowRight_max', 'cursor_position_diff_Leftclick_max', 'cursor_position_diff_ArrowLeft_max', 'cursor_position_diff_dot_max', 'cursor_position_diff_ArrowDown_max', 'cursor_position_diff_ArrowUp_max', 'cursor_position_diff_Enter_max', 'cursor_position_diff_Delete_max', 'word_count_diff_q_std', 'word_count_diff_Space_std', 'word_count_diff_Backspace_std', 'word_count_diff_Leftclick_std', 'word_count_diff_dot_std', 'word_count_diff_comma_std', 'word_count_diff_Enter_std', 'word_count_diff_abostrove_std', 'word_count_diff_Delete_std', 'word_count_diff_q_mean', 'word_count_diff_Space_mean', 'word_count_diff_Backspace_mean', 'word_count_diff_Leftclick_mean', 'word_count_diff_dot_mean', 'word_count_diff_comma_mean', 'word_count_diff_Enter_mean', 'word_count_diff_abostrove_mean', 'word_count_diff_Delete_mean', 'word_count_diff_q_sum', 'word_count_diff_Space_sum', 'word_count_diff_Backspace_sum', 'word_count_diff_Leftclick_sum', 'word_count_diff_dot_sum', 'word_count_diff_comma_sum', 'word_count_diff_Enter_sum', 'word_count_diff_abostrove_sum', 'word_count_diff_Delete_sum', 'word_count_diff_Space_median', 'word_count_diff_Delete_median', 'word_count_diff_Space_max', 'word_count_diff_Leftclick_max', 'word_count_diff_Delete_max', 'time_diff_Input_std', 'time_diff_Remove/Cut_std', 'time_diff_Nonproduction_std', 'time_diff_Replace_std', 'time_diff_Paste_std', 'time_diff_Input_mean', 'time_diff_Remove/Cut_mean', 'time_diff_Nonproduction_mean', 'time_diff_Replace_mean', 'time_diff_Paste_mean', 'time_diff_Input_sum', 'time_diff_Remove/Cut_sum', 'time_diff_Nonproduction_sum', 'time_diff_Replace_sum', 'time_diff_Paste_sum', 'time_diff_Input_median', 'time_diff_Remove/Cut_median', 'time_diff_Nonproduction_median', 'time_diff_Replace_median', 'time_diff_Paste_median', 'time_diff_Input_max', 'time_diff_Remove/Cut_max', 'time_diff_Nonproduction_max', 'time_diff_Replace_max', 'time_diff_Paste_max', 'cursor_position_diff_Input_std', 'cursor_position_diff_Remove/Cut_std', 'cursor_position_diff_Nonproduction_std', 'cursor_position_diff_Replace_std', 'cursor_position_diff_Paste_std', 'cursor_position_diff_Input_mean', 'cursor_position_diff_Remove/Cut_mean', 'cursor_position_diff_Nonproduction_mean', 'cursor_position_diff_Replace_mean', 'cursor_position_diff_Paste_mean', 'cursor_position_diff_Input_sum', 'cursor_position_diff_Remove/Cut_sum', 'cursor_position_diff_Nonproduction_sum', 'cursor_position_diff_Replace_sum', 'cursor_position_diff_Paste_sum', 'cursor_position_diff_Remove/Cut_median', 'cursor_position_diff_Nonproduction_median', 'cursor_position_diff_Replace_median', 'cursor_position_diff_Paste_median', 'cursor_position_diff_Input_max', 'cursor_position_diff_Remove/Cut_max', 'cursor_position_diff_Nonproduction_max', 'cursor_position_diff_Replace_max', 'cursor_position_diff_Paste_max', 'word_count_diff_Input_std', 'word_count_diff_Remove/Cut_std', 'word_count_diff_Replace_std', 'word_count_diff_Paste_std', 'word_count_diff_Input_mean', 'word_count_diff_Remove/Cut_mean', 'word_count_diff_Replace_mean', 'word_count_diff_Paste_mean', 'word_count_diff_Input_sum', 'word_count_diff_Remove/Cut_sum', 'word_count_diff_Replace_sum', 'word_count_diff_Paste_sum', 'word_count_diff_Replace_median', 'word_count_diff_Paste_median', 'word_count_diff_Replace_max', 'word_count_diff_Paste_max', 'word_len_count', 'word_len_mean', 'word_len_std', 'word_len_min', 'word_len_max', 'word_len_first', 'word_len_last', 'word_len_sem', 'word_len_q1', 'word_len_median', 'word_len_q3', 'word_len_skew', 'word_len_kurt', 'word_len_sum', 'word_len_ge_2_count', 'word_len_ge_3_count', 'word_len_ge_4_count', 'word_len_ge_5_count', 'word_len_ge_6_count', 'word_len_ge_7_count', 'word_len_ge_8_count', 'word_len_ge_9_count', 'word_len_ge_10_count', 'word_len_ge_11_count', 'word_len_ge_12_count', 'word_len_ge_13_count', 'word_len_ge_14_count', 'word_len_ge_15_count', 'sent_count', 'sent_len_mean', 'sent_len_std', 'sent_len_min', 'sent_len_max', 'sent_len_first', 'sent_len_last', 'sent_len_sem', 'sent_len_q1', 'sent_len_median', 'sent_len_q3', 'sent_len_skew', 'sent_len_kurt', 'sent_len_sum', 'sent_word_count_mean', 'sent_word_count_std', 'sent_word_count_min', 'sent_word_count_max', 'sent_word_count_first', 'sent_word_count_last', 'sent_word_count_sem', 'sent_word_count_q1', 'sent_word_count_median', 'sent_word_count_q3', 'sent_word_count_skew', 'sent_word_count_kurt', 'sent_word_count_sum', 'sent_len_ge_15_count', 'sent_len_ge_50_count', 'sent_len_ge_60_count', 'sent_len_ge_75_count', 'sent_len_ge_80_count', 'sent_len_ge_120_count', 'sent_len_ge_150_count', 'sent_len_ge_180_count', 'sent_len_ge_100_count', 'paragraph_count', 'paragraph_len_mean', 'paragraph_len_std', 'paragraph_len_min', 'paragraph_len_max', 'paragraph_len_first', 'paragraph_len_last', 'paragraph_len_sem', 'paragraph_len_q1', 'paragraph_len_median', 'paragraph_len_q3', 'paragraph_len_skew', 'paragraph_len_kurt', 'paragraph_len_sum', 'paragraph_word_count_mean', 'paragraph_word_count_std', 'paragraph_word_count_min', 'paragraph_word_count_max', 'paragraph_word_count_first', 'paragraph_word_count_last', 'paragraph_word_count_sem', 'paragraph_word_count_q1', 'paragraph_word_count_median', 'paragraph_word_count_q3', 'paragraph_word_count_skew', 'paragraph_word_count_kurt', 'paragraph_word_count_sum', 'word_len_ge_6_count_mp', 'word_len_ge_6_count_mt', 'word_len_ge_6_count_mi', 'word_len_ge_6_count_md', 'initial_pause_time', 'total_time', 'IKI_mean', 'IKI_median', 'IKI_max', 'IKI_std', 'IKI_sum', 'IKI_skew', 'IKI_bw_letter_mean', 'IKI_bw_letter_sum', 'IKI_bw_letter_max', 'IKI_bw_letter_min', 'IKI_bw_letter_skew', 'Total_Number_of_pause', 'Number_of_Pause_bw_word', 'Number_of_backspaces', 'backspacing_time_mean', 'backspacing_time_std', 'backspacing_time_max', 'backspacing_time_sum', 'backspacing_time_skew', 'backspacing_time_count', 'long_pauses_count', 'long_pauses_sum', 'long_pauses_std', 'long_pauses_mean', 'percentage_long_pauses', 'Number of Revesions', 'Number of revisions Backspace 2 word', 'Number of revisions Remove/Cut 2 word', 'Number of revisions Backspace 3 word', 'Number of revisions Remove/Cut 3 word', 'Number of revisions Backspace 4 word', 'Number of revisions Remove/Cut 4 word', 'Number of revisions Backspace 5 word', 'Number of revisions Remove/Cut 5 word', 'Number of revisions Backspace 10 word', 'Number of revisions Remove/Cut 10 word', 'Number of revisions Backspace 14 word', 'Number of revisions Remove/Cut 14 word', 'Number of revisions Backspace 50 word', 'Number of revisions Remove/Cut 50 word', 'leading-edge revisions_max', 'leading-edge revisions_sum', 'leading-edge revisions_std', 'leading-edge revisions_mean', 'leading-edge revisions_count', 'sequence_of_backspaces_mean', 'sequence_of_backspaces_std', 'sequence_of_backspaces_sum', 'sequence_of_backspaces_max', 'sequence_of_backspaces_min', 'number_of_characters', 'number_of_characters_at_leading_edg', 'num_bursts', 'max_chars_per_burst', 'mean_chars_per_burst', 'std_chars_per_burst', 'num_revision_bursts', 'Percentage_of_R_bursts', 'num_insertion_bursts', 'Percentage_of_I_bursts', 'sd_keystrokes_5s', 'mean_keystrokes_5s', 'max_keystrokes_5s', 'min_keystrokes_5s', 'median_keystrokes_5s', 'sd_keystrokes_8s', 'mean_keystrokes_8s', 'max_keystrokes_8s', 'min_keystrokes_8s', 'median_keystrokes_8s', 'sd_keystrokes_10s', 'mean_keystrokes_10s', 'max_keystrokes_10s', 'min_keystrokes_10s', 'median_keystrokes_10s', 'sd_keystrokes_15s', 'mean_keystrokes_15s', 'max_keystrokes_15s', 'min_keystrokes_15s', 'median_keystrokes_15s', 'sd_keystrokes_30s', 'mean_keystrokes_30s', 'max_keystrokes_30s', 'min_keystrokes_30s', 'median_keystrokes_30s', 'sd_keystrokes_45s', 'mean_keystrokes_45s', 'max_keystrokes_45s', 'min_keystrokes_45s', 'median_keystrokes_45s', 'sd_keystrokes_60s', 'mean_keystrokes_60s', 'max_keystrokes_60s', 'min_keystrokes_60s', 'median_keystrokes_60s', 'sd_keystrokes_80s', 'mean_keystrokes_80s', 'max_keystrokes_80s', 'min_keystrokes_80s', 'median_keystrokes_80s', 'sd_keystrokes_120s', 'mean_keystrokes_120s', 'max_keystrokes_120s', 'min_keystrokes_120s', 'median_keystrokes_120s', 'sd_keystrokes_180s', 'mean_keystrokes_180s', 'max_keystrokes_180s', 'min_keystrokes_180s', 'median_keystrokes_180s', 'slope_keystrokes_4s', 'slope_keystrokes_5s', 'slope_keystrokes_8s', 'slope_keystrokes_10s', 'slope_keystrokes_15s', 'slope_keystrokes_20s', 'slope_keystrokes_25s', 'slope_keystrokes_30s', 'slope_keystrokes_60s', 'slope_keystrokes_80s', 'entropy_keystrokes_5s', 'entropy_keystrokes_8s', 'entropy_keystrokes_10s', 'entropy_keystrokes_15s', 'entropy_keystrokes_25s', 'entropy_keystrokes_30s', 'entropy_keystrokes_45s', 'entropy_keystrokes_60s', '3s_extrema_per_user', '5s_extrema_per_user', '8s_extrema_per_user', '10s_extrema_per_user', '15s_extrema_per_user', '30s_extrema_per_user', '40s_extrema_per_user', '50s_extrema_per_user', '60s_extrema_per_user', '3_distance_stats_mean', '3_distance_stats_std', '3_distance_stats_sum', '3_distance_stats_max', '5_distance_stats_sum', '8_distance_stats_sum', '10_distance_stats_mean', '10_distance_stats_std', '10_distance_stats_sum', '10_distance_stats_max', '15_distance_stats_mean', '15_distance_stats_std', '15_distance_stats_sum', '15_distance_stats_max', '20_distance_stats_mean', '20_distance_stats_std', '20_distance_stats_sum', '20_distance_stats_max', '30_distance_stats_mean', '30_distance_stats_std', '30_distance_stats_sum', '30_distance_stats_max', '40_distance_stats_mean', '40_distance_stats_std', '40_distance_stats_sum', '40_distance_stats_max', '50_distance_stats_mean', '50_distance_stats_std', '50_distance_stats_sum', '50_distance_stats_max', '60_distance_stats_mean', '60_distance_stats_std', '60_distance_stats_sum', '60_distance_stats_max', '80_distance_stats_mean', '80_distance_stats_std', '80_distance_stats_sum', '80_distance_stats_max', 'sent_len_ge_50_count_IKI_ median_Divid', 'sent_word_count_mean_paragraph_word_count_mean_Divid', 'down_event_8_cnt_pauses_1_sec_Divid', 'word_count_diff_Input_mean_sent_len_std_Divid', 'word_count_diff_Input_mean_paragraph_word_count_mean_Divid', 'IKI_ median_sent_len_std_Divid', 'paragraph_word_count_mean_pauses_1_sec_Divid', 'word_len_max_IKI_ median_Divid', 'word_len_max_pauses_1_sec_Divid', 'input_word_length_mean_IKI_ median_Divid', 'sent_len_ge_50_count_time_diff_Input_mean_Div', 'P-bursts_count_time_diff_Input_mean_Div', 'pauses_1_sec_P-bursts_count_Divv', 'text_change_4_cnt_sent_len_ge_50_count_Div', 'down_event_5_cnt_sent_len_ge_50_count_Div', 'P-bursts_median_sent_len_ge_50_count_Div', 'sent_len_std_time_diff_Backspace_median_Div', 'text_change_3_cnt_time_diff_Backspace_median_Div', 'sent_len_median_time_diff_Backspace_median_Div', 'sent_len_median_paragraph_count_Div', 'word_count_diff_Input_mean_sent_len_median_Div', 'word_count_diff_Input_mean_text_change_3_cnt_Div', 'word_count_diff_Input_mean_word_len_max_Div', 'word_count_diff_Input_mean_paragraph_count_Div', 'text_change_3_cnt_paragraph_len_last_Div', 'word_len_std_word_count_diff_Input_mean_MUl', 'word_len_ge_5_count_IKI_median_MUl', 'time_diff_Remove/Cut_median_word_len_std_DIVV', 'text_change_3_cnt_punct_cnt_DIVV', 'word_len_ge_9_count_word_len_ge_10_count_SUBB', 'word_len_ge_9_count_word_len_ge_10_count_MULT', 'word_len_ge_8_count_word_len_ge_9_count_SUBB', 'word_len_ge_8_count_word_len_ge_9_count_MULT', 'word_len_ge_7_count_word_len_ge_8_count_SUBB', 'word_len_ge_7_count_word_len_ge_8_count_MULT', 'word_len_ge_6_count_word_len_ge_7_count_SUBB', 'word_len_ge_6_count_word_len_ge_7_count_MULT', 'word_len_ge_5_count_word_len_ge_6_count_SUBB', 'word_len_ge_5_count_word_len_ge_6_count_MULT', 'word_len_ge_4_count_word_len_ge_5_count_SUBB', 'word_len_ge_4_count_word_len_ge_5_count_MULT', 'pauses_1_sec_pauses_3_sec_DIVV', 'pauses_1_sec_pauses_3_sec_SUB', 'pauses_2_sec_pauses_3_sec_DIVV', 'pauses_2_sec_pauses_3_sec_SUB', 'pauses_2_sec_pauses_1_sec_DIVV', 'pauses_2_sec_pauses_1_sec_SUB', 'IKI_median_sent_len_ge_80_count_MULT', 'IKI_median_change_3_cnt_MULT', 'IKI_median_word_len_std_MULT', 'IKI_median_paragraph_len_max_MULT', 'IKI_median_pauses_half_sec_MULT', 'pauses_half_sec_word_len_mean_MULT', 'pauses_half_sec_sent_word_count_first_MULT', 'word_len_sem_pauses_1_half_sec_MULT', 'word_len_sem_text_change_3_cnt_MULT', 'word_len_sem_sent_len_ge_75_count_MULT', 'inter_key_median_lantency_sent_len_ge_80_count_MULT', 'inter_key_median_lantency_text_change_3_cnt_MULT', 'sd_keystrokes_180s_sent_word_count_max_DIV', 'sd_keystrokes_180s_mean_keystrokes_5s_DIV', 'sd_keystrokes_180s_paragraph_word_count_q1_DIV', 'sd_keystrokes_180s_60s_extrema_per_user_DIV', 'Percentage_of_I_bursts_activity_2_cnt_DIVV', 'Percentage_of_I_bursts_sent_len_median_DIVV', 'Percentage_of_I_bursts_paragraph_word_count_median_DIVV']

In [78]:
not_include_feature = [col for col in feature_selected if train_feats[col].value_counts().isin(np.arange(2470, 2000,-1)).any()]
feature_names = [col for col in feature_selected if col not in not_include_feature]
# # Na
na_values = [col for col in feature_names if train_feats[col].isna().sum() > 2000]
feature_names = [col for col in feature_names if col not in na_values]
print('FEATURES : ', len(feature_names))
feature_selected= feature_names

FEATURES :  678


In [79]:
# MODEL_SELECTION = ['xgboost']

MODEL_SELECTION = ['catboost','xgboost', 'lightgbm','rfr','svr', 'ridge']

MODELS = [] 
weights = []

# Blending, CV

In [80]:
# test_feats = train_df.copy()
# train_feats = train_df.copy()

In [81]:
ycol = 'score'

feature_names = None

oof = []
prediction = test_feats[['id']]
scores = dict()
test_preds = dict()

prediction[ycol] = 0
df_importance_list = []
skfold = StratifiedKFold(n_splits=10,shuffle=True, random_state=42)

for select in MODEL_SELECTION:
    
    print()
    local_oof = []
    test_predict = np.zeros(len(test_feats))
    
    if select == 'rfr':
        
        print('########### Random Forest')
        model = rfr_model
        feature_names  = feature_selected
        # Apply Trasformation for Smoothing
        transformation('No_transfomer')

    elif select =='svr':
        
        print('########### SVM ')
        model = svr_model
        feature_names = svr_features
        # Apply Trasformation for Smoothing
        transformation('MinMax')

    elif select =='ridge':
        model = ridge_model
        print('########### Ridge')
        
        feature_names = ridge_features
        
        # Apply Trasformation for Smoothing
        transformation('MinMax')

    elif select == 'lightgbm' :
        model = lgbm_model
        print('########### LightGBM ')
        
        feature_names = feature_selected
        transformation('StanderScaler')
                
    elif select == 'catboost':
        model = cbt_model
        print('########### CatBoost')
        
        feature_names = feature_selected
        transformation('StanderScaler')

    elif select == 'xgboost':
        model = xgb_model
        print('########### Xgboost')
        
        feature_names = feature_selected
        transformation('StanderScaler')
        
    print('FEATURES TRAINING : ', len(feature_names))
    for fold_id, (trn_idx, val_idx) in enumerate(skfold.split(X=train_feats[feature_names], y=train_feats['score'].apply(lambda x:x*10).astype(int))):

            X_train = train_feats.iloc[trn_idx][feature_names].fillna(0)
            Y_train = train_feats.iloc[trn_idx][ycol]

            X_val = train_feats.iloc[val_idx][feature_names].fillna(0)
            Y_val = train_feats.iloc[val_idx][ycol]

            fit_model = model
            
            print('\nFold_{} Training ================================\n'.format(fold_id+1))

            if select == 'lightgbm':
                fit_model.fit(X_train,Y_train,eval_names=['train', 'valid'],eval_set=[(X_train, Y_train), (X_val, Y_val)],verbose=-1,eval_metric='rmse',early_stopping_rounds=100)
                
            elif select == 'xgboost':
                fit_model.fit(X_train,Y_train,
                              eval_set=[(X_train, Y_train), (X_val, Y_val)],
                              verbose=0,
                              early_stopping_rounds=50)

            elif select == 'catboost':
                fit_model.fit(X_train,Y_train,
                              eval_set=[(X_train, Y_train), (X_val, Y_val)],
                              verbose=0,
                              early_stopping_rounds=50)
            else:
                fit_model.fit(X_train, Y_train)
                
            pred_val = fit_model.predict(X_val)

            df_oof = train_feats.iloc[val_idx][['id', ycol]].copy()
            df_oof['pred'] = pred_val
            
            oof.append(df_oof)
            local_oof.append(df_oof)
            
            pred_test = fit_model.predict(test_feats[feature_names].fillna(0))
            
            test_predict += pred_test / skfold.n_splits
#             prediction[ycol] += pred_test / (skfold.n_splits * len(MODEL_SELECTION))
            
            del pred_val, X_train, Y_train, X_val, Y_val, pred_test
     
    MODELS.append((select, fit_model))
    
    del model, fit_model
    gc.collect()
    
    df_oof_local = pd.concat(local_oof)
    scores[select] = df_oof_local.sort_index()['pred'].values
    test_preds[select] = test_predict
    
    rmse = mean_squared_error(df_oof_local[ycol], np.clip(df_oof_local['pred'], a_min=0.5, a_max=6.0), squared=False)
    weights.append(rmse)
    print(f'{select} rmse:', rmse.round(3))


########### CatBoost
FEATURES TRAINING :  678

Fold_1 Training ================================


Fold_2 Training ================================


Fold_3 Training ================================


Fold_4 Training ================================


Fold_5 Training ================================


Fold_6 Training ================================


Fold_7 Training ================================


Fold_8 Training ================================


Fold_9 Training ================================


Fold_10 Training ================================

catboost rmse: 0.602

########### Xgboost
FEATURES TRAINING :  678

Fold_1 Training ================================


Fold_2 Training ================================


Fold_3 Training ================================


Fold_4 Training ================================


Fold_5 Training ================================


Fold_6 Training ================================


Fold_7 Training ================================


Fold_8 Training =

In [82]:
df_oof = pd.concat(oof)
rmse = mean_squared_error(df_oof[ycol], np.clip(df_oof['pred'], a_min=0.5, a_max=6.0), squared=False)
print('AVR RMSE :', rmse.round(4))

AVR RMSE : 0.6006


# Manual Ensemble

In [83]:
# display(prediction)
# prediction['score'] = np.clip(prediction['score'], a_min=0.5, a_max=6.0)
# prediction.to_csv('submission.csv', index=False)

In [84]:
# MODELS

In [85]:
model_weight = [('catboost' ,0.08),
                ('xgboost'  ,0.05),
                ('lightgbm' ,0.10),
                ('rfr'      ,0.05),
                ('svr'      ,0.36),
                ('ridge'    ,0.36)
               ]

valid = np.zeros(len(train_feats))
test_y = np.zeros(len(test_feats))
confi = 0
for model_name, weight in model_weight:
    confi += weight
    scr = scores[model_name] * weight
    valid += scr
    
    tcr = test_preds[model_name] * weight
    test_y += tcr
print(confi)
ensemble_rmse = mean_squared_error(train_feats['score'],np.clip(valid, a_min=0.5, a_max=6.0) , squared=False)
print("ENSEMBLE RMSE : ", ensemble_rmse.round(4))

1.0
ENSEMBLE RMSE :  0.58


In [86]:
test_y = np.clip(test_y, a_min=0.5, a_max=6.0)

submission = pd.DataFrame({'id': test_feats['id'], 'score': test_y})
submission.to_csv('submission.csv', index=False)
display(submission)

,id,score
0,0000aaaa,0.5
1,2222bbbb,0.5
2,4444cccc,0.5


# Ensembleing (VotingRegressor)

In [87]:
# try:
# #     weights = evaluate_models(MODELS, train_feats[svr_features].fillna(0), train_feats['score'])
#     # Use the weights (scores) as a weighting for the ensemble
#     print(weights)
#     ensemble = VotingRegressor(estimators=MODELS, weights=weights)
#     ensemble.fit(train_feats[feature_selected].fillna(0), train_feats['score'])
#     test_y = ensemble.predict(test_feats[feature_selected].fillna(0))
#     print(test_y)
# except Exception as e: 
#     print(e)

In [88]:
# print('Submite to host ..')
# submission = pd.DataFrame({'id': test_feats['id'], 'score': test_y})
# submission.to_csv('submission.csv', index=False)
# display(submission)

# Stacking

In [89]:
stacked = pd.DataFrame(scores)
stacked['score'] = train_feats['score']
stacked['id'] = train_feats.id
stacked.head()

,catboost,xgboost,lightgbm,rfr,svr,ridge,score,id
0,3.662747,3.687229,3.652883,3.527932,3.789026,3.760931,3.5,001519c8
1,3.464752,3.463559,3.361107,3.365100,3.477784,3.617423,3.5,0022f953
2,4.589428,4.646332,4.583335,4.612864,4.538848,4.516136,6.0,0042269b
3,2.532401,2.531767,2.665262,2.521026,2.651385,2.899786,2.0,0059420b
4,3.230374,3.331836,3.390294,3.244797,3.362864,3.549745,4.0,0075873a


In [90]:
test_stacked = pd.DataFrame(test_preds)
test_stacked['id'] = test_feats.id
test_stacked.head()

,catboost,xgboost,lightgbm,rfr,svr,ridge,id
0,3.190724,3.369513,3.439322,1.731185,3.092078,-802.163227,0000aaaa
1,2.063074,2.057062,1.993619,1.710873,3.092078,-1306.286022,2222bbbb
2,1.981021,1.974813,1.984451,1.647943,3.092078,-1097.342539,4444cccc


In [91]:
ycol = 'score'
feature_names = MODEL_SELECTION

model = ridge_model

oof = []
prediction = test_feats[['id']]
prediction[ycol] = 0
df_importance_list = []

skfold = StratifiedKFold(n_splits=10,shuffle=True, random_state=42)

for fold_id, (trn_idx, val_idx) in enumerate(skfold.split(X=stacked[feature_names], y=stacked['score'].apply(lambda x:x*10).astype(int))):
    X_train = stacked.iloc[trn_idx][feature_names]
    Y_train = stacked.iloc[trn_idx][ycol]

    X_val = stacked.iloc[val_idx][feature_names]
    Y_val = stacked.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    model.fit(X_train, Y_train)

    pred_val = model.predict(X_val)
    
    df_oof = stacked.iloc[val_idx][['id', ycol]].copy()
    df_oof['pred'] = pred_val
    oof.append(df_oof)

    pred_test = model.predict(test_stacked[feature_names])
    
    prediction[ycol] += pred_test / skfold.n_splits

#     df_importance = pd.DataFrame({
#         'column': feature_names,
#         'importance': lgb_model.feature_importances_,
#     })
#     df_importance_list.append(df_importance)

    del pred_val, X_train, Y_train, X_val, Y_val
    gc.collect()
    
    
# df_importance = pd.concat(df_importance_list)
# df_importance = df_importance.groupby(['column'])['importance'].agg(
#     'mean').sort_values(ascending=False).reset_index()
# df_importance


Fold_1 Training ================================


Fold_2 Training ================================


Fold_3 Training ================================


Fold_4 Training ================================


Fold_5 Training ================================


Fold_6 Training ================================


Fold_7 Training ================================


Fold_8 Training ================================


Fold_9 Training ================================


Fold_10 Training ================================



In [92]:
df_oof = pd.concat(oof)
rmse = mean_squared_error(df_oof[ycol], np.clip(df_oof['pred'], a_min=0.5, a_max=6.0), squared=False)
print('rmse:', rmse)

rmse: 0.5793225882385351


In [93]:
# print('Submite to host ..')
# display(prediction)
# prediction['score'] = np.clip(prediction['score'], a_min=0.5, a_max=6.0)
# prediction.to_csv('submission.csv', index=False)

# Stack + Ensemble

In [94]:
stack_valid = df_oof['pred'].sort_index().values

In [95]:
StEn = valid * 0.5 + stack_valid * 0.5
STEnTest = test_y * 0.5 +  prediction['score'] * 0.5
strmse = mean_squared_error(train_feats.score, StEn, squared=False)
print("Stack&Ensemble RMSE : ", strmse.round(3))

Stack&Ensemble RMSE :  0.579


In [96]:
# print('Submite to host ..')
# STEnTest = np.clip(STEnTest, a_min=0.5, a_max=6.0)
# submission = pd.DataFrame({'id': test_feats['id'], 'score': STEnTest})
# submission.to_csv('submission.csv', index=False)
# display(submission)